In [ ]:
#!pip install seaborn

In [ ]:
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
import json
import geopandas as gpd
import warnings
warnings.filterwarnings("ignore")
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from datetime import datetime
from shapely.geometry import Point
#import plotly.express as px

In [ ]:
permits = pd.read_csv("G:/Independent Study/Street_Construction_Permits__2022-Present_.csv")
permits.head()

,PermitNumber,ApplicationTrackingID,SequenceNumber,ApplicationTypeShortDesc,PermitStatusID,PermitStatusShortDesc,PermitSeriesID,PermitSeriesShortDesc,PermitTypeID,PermitTypeDesc,PermitNumberOfZones,PermitLinearFeet,PermitTotalSqFeet,PermitEstimatedNumberOfCuts,EquipmentTypeDesc,NumberOfContainers,NumberOfMiniContainers,SpecificStipulations,PreviousPermitNumber,NextPermitNumber,EmergencyIssueDate,PermitIssueDate,IssuedWorkStartDate,IssuedWorkEndDate,BoroughName,PermitHouseNumber,OnStreetName,FromStreetName,ToStreetName,PermitteeName,PermitPurposeComments,PermitLocationComments,PavementShortDesc,SideWalkShortDesc,CreatedOn,ModifiedOn,OFTCode
0,B002022287A02,2.022101e+15,1.0,Renew,006,ISSUED & PRINTED,0,MISCELLANEOUS CASH RECEIPTS,0071,MISCELLANEOUS - EMBARGO WAIVER REQUEST,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,10/14/2022 09:32:12 AM,NaN,NaN,BROOKLYN,NaN,NaN,NaN,NaN,"ZDG, LLC",A RECEIPT OF PAYMENT FOR A HOLIDAY EMBARGO WAIVER,NaN,NaN,NaN,10/14/2022 09:32:12 AM,10/14/2022 09:32:12 AM,NaN
1,B002022325A01,2.022112e+15,1.0,Emergency,006,ISSUED & PRINTED,0,MISCELLANEOUS CASH RECEIPTS,0071,MISCELLANEOUS - EMBARGO WAIVER REQUEST,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,11/21/2022 08:55:25 AM,NaN,NaN,BROOKLYN,NaN,NaN,NaN,NaN,CONSOLIDATED EDISON,A RECEIPT OF PAYMENT FOR A HOLIDAY EMBARGO WAIVER,NaN,NaN,NaN,11/21/2022 08:55:24 AM,11/21/2022 08:55:24 AM,NaN
2,B002022339A03,2.022121e+15,1.0,Emergency,006,ISSUED & PRINTED,0,MISCELLANEOUS CASH RECEIPTS,0071,MISCELLANEOUS - EMBARGO WAIVER REQUEST,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,12/05/2022 10:33:09 PM,NaN,NaN,BROOKLYN,NaN,NaN,NaN,NaN,CONSOLIDATED EDISON,A RECEIPT OF PAYMENT FOR A HOLIDAY EMBARGO WAIVER,NaN,NaN,NaN,12/05/2022 10:33:09 PM,12/05/2022 10:33:09 PM,NaN
3,B012018331F56,2.018112e+15,2.0,New,030,EXPIRED,1,STREET OPENING PERMIT,0121P,CONSTRUCT OR ALTER MANHOLE &/OR CASTING,1.0,10.0,NaN,NaN,NaN,NaN,NaN,JN,NaN,B012018355C44,NaN,11/27/2018 02:56:38 PM,12/03/2018 12:00:00 AM,12/31/2018 11:59:59 PM,BROOKLYN,NaN,77 STREET,4 AVENUE,5 AVENUE,"TC SYSTEMS, INC",NaN,VZMH 003 (BRK 09_018) (T),NaN,CONCRETE,11/27/2018 02:56:38 PM,01/01/2022 12:05:34 AM,310430305430305600.0
4,B012018362B39,2.018123e+15,1.0,New,030,EXPIRED,1,STREET OPENING PERMIT,0114,REPAIR SEWER,1.0,25.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,12/28/2018 03:22:57 PM,12/31/2018 12:00:00 AM,01/29/2019 11:59:59 PM,BROOKLYN,1274,EAST 52 STREET,AVENUE I,AVENUE J,G. ROLLINS PLUMBING & HEATING LLC,REPAIR SEWER,NaN,ASPHALT,CONCRETE,12/28/2018 03:22:57 PM,01/30/2022 12:05:30 AM,337080314030314112.0


In [ ]:
permits.shape

(1201732, 37)

### Check Utility type
- Get Electricity and Gas into two dataframes for easier processing

In [ ]:
permits['PermitTypeDesc'].value_counts()

PLACE EQUIPMENT OTHER THAN CRANE OR SHOV              143888
COMMERCIAL REFUSE CONTAINER                            81520
REPAIR GAS                                             68059
MAJOR INSTALLATION - GAS                               54890
D.D.C. CONTRACTOR MAJOR RECONSTRUCTION                 51981
OCCUPANCY OF ROADWAY AS STIPULATED                     45107
OCCUPANCY OF SIDEWALK AS STIPULATED                    44294
REPAIR GAS - PROTECTED                                 38387
PLACE MATERIAL ON STREET                               27801
REPAIR SIDEWALK                                        27356
MAJOR INSTALLATIONS - GAS-PROTECTED                    21507
PLACE OPEN CONSTRUCTION DEBRIS CONTAINER ON STREET     20542
PED RAMP MODIFY/INSTALL GOVERNMENT                     20076
CROSSING SIDEWALK                                      19035
COMMERCIAL REFUSE CONTAINER CRITICAL STS               18728
TEMP. CONST. SIGNS/MARKINGS                            17426
UTILITY MANHOLE EMBARGO 

In [ ]:
permits_gas = permits[permits['PermitTypeDesc'].str.contains('GAS')]
permits_gas.head()

,PermitNumber,ApplicationTrackingID,SequenceNumber,ApplicationTypeShortDesc,PermitStatusID,PermitStatusShortDesc,PermitSeriesID,PermitSeriesShortDesc,PermitTypeID,PermitTypeDesc,PermitNumberOfZones,PermitLinearFeet,PermitTotalSqFeet,PermitEstimatedNumberOfCuts,EquipmentTypeDesc,NumberOfContainers,NumberOfMiniContainers,SpecificStipulations,PreviousPermitNumber,NextPermitNumber,EmergencyIssueDate,PermitIssueDate,IssuedWorkStartDate,IssuedWorkEndDate,BoroughName,PermitHouseNumber,OnStreetName,FromStreetName,ToStreetName,PermitteeName,PermitPurposeComments,PermitLocationComments,PavementShortDesc,SideWalkShortDesc,CreatedOn,ModifiedOn,OFTCode
107,B012023089A25,2.023033e+15,2.0,New,006,ISSUED & PRINTED,1,STREET OPENING PERMIT,0103,MAJOR INSTALLATION - GAS,1.0,75.0,NaN,NaN,NaN,NaN,NaN,DP,NaN,NaN,NaN,03/30/2023 06:28:03 AM,03/30/2023 06:28:03 AM,06/27/2023 11:59:59 PM,BROOKLYN,NaN,AVENUE M,EAST 35 STREET,NaN,THE BROOKLYN UNION GAS COMPANY D/B/A NATIONAL ...,MAJOR INSTALLATION - GAS,SEE SKETCH,ASPHALT,CONCRETE,03/30/2023 06:28:02 AM,03/30/2023 06:28:02 AM,NaN
238,M012019112A85,2.019042e+15,1.0,New,030,EXPIRED,1,STREET OPENING PERMIT,0103P,MAJOR INSTALLATIONS - GAS-PROTECTED,1.0,4.0,NaN,NaN,NaN,NaN,NaN,GC,NaN,NaN,NaN,04/22/2019 09:59:18 AM,04/22/2019 09:59:18 AM,07/17/2019 11:59:59 PM,MANHATTAN,NaN,3 AVENUE,EAST 82 STREET,EAST 83 STREET,CONSOLIDATED EDISON,DEMO,94 FT SSC OF E 83RD ST &amp; 9 FT WEC OF 3RD AVE,ASPHALT,CONCRETE,04/22/2019 09:59:18 AM,07/18/2022 12:05:20 AM,110210118630118656.0
242,M012019121A35,2.019043e+15,1.0,New,030,EXPIRED,1,STREET OPENING PERMIT,0103,MAJOR INSTALLATION - GAS,1.0,25.0,NaN,NaN,NaN,NaN,NaN,GC,NaN,NaN,NaN,05/01/2019 10:12:46 AM,05/01/2019 10:12:46 AM,07/29/2019 11:59:59 PM,MANHATTAN,NaN,1 AVENUE,EAST 66 STREET,EAST 67 STREET,CONSOLIDATED EDISON,Install Gas Svc,109' SSC of E 67 St &amp; 3' EEC of 1 Ave,ASPHALT,CONCRETE,05/01/2019 10:12:46 AM,07/30/2022 12:05:21 AM,110010118310118336.0
251,M012019162B93,2.019061e+15,1.0,New,030,EXPIRED,1,STREET OPENING PERMIT,0103,MAJOR INSTALLATION - GAS,1.0,4.0,NaN,NaN,NaN,NaN,NaN,GC,NaN,NaN,NaN,06/11/2019 02:52:08 PM,06/11/2019 02:52:08 PM,09/07/2019 11:59:59 PM,MANHATTAN,NaN,EAST 86 STREET,1 AVENUE,2 AVENUE,CONSOLIDATED EDISON,Demo,128 FT EEC OF 2ND AVE &amp; 10 FT SSC OF E 86 ST,ASPHALT,CONCRETE,06/11/2019 02:52:08 PM,09/08/2022 12:05:22 AM,118710110010110112.0
252,M012019163A41,2.019061e+15,1.0,Emergency,030,EXPIRED,1,STREET OPENING PERMIT,0122,REPAIR GAS,1.0,4.0,NaN,NaN,NaN,NaN,NaN,GC,NaN,NaN,06/12/2019 11:06:55 AM,06/13/2019 09:38:16 AM,06/12/2019 11:06:55 AM,07/11/2019 11:59:59 PM,MANHATTAN,NaN,WEST 83 STREET,CENTRAL PARK WEST,COLUMBUS AVENUE,CONSOLIDATED EDISON,Repair Gas,365' wwc of cpw &amp; 1' nsc of w 83,ASPHALT,CONCRETE,06/12/2019 11:06:56 AM,07/12/2022 12:05:16 AM,135430114510115712.0


In [ ]:
permits_gas.shape[0]/permits.shape[0]*100

16.140870010950863

In [ ]:
cols = ["PermitNumber","OFTCode","OnStreetName","FromStreetName","ToStreetName", "BoroughName","PermitIssueDate","PermitLinearFeet", "ApplicationTypeShortDesc", "PermitSeriesShortDesc"]
permits_gas = permits_gas[cols]
permits_gas.head()

,PermitNumber,OFTCode,OnStreetName,FromStreetName,ToStreetName,BoroughName,PermitIssueDate,PermitLinearFeet,ApplicationTypeShortDesc,PermitSeriesShortDesc
107,B012023089A25,NaN,AVENUE M,EAST 35 STREET,NaN,BROOKLYN,03/30/2023 06:28:03 AM,75.0,New,STREET OPENING PERMIT
238,M012019112A85,110210118630118656.0,3 AVENUE,EAST 82 STREET,EAST 83 STREET,MANHATTAN,04/22/2019 09:59:18 AM,4.0,New,STREET OPENING PERMIT
242,M012019121A35,110010118310118336.0,1 AVENUE,EAST 66 STREET,EAST 67 STREET,MANHATTAN,05/01/2019 10:12:46 AM,25.0,New,STREET OPENING PERMIT
251,M012019162B93,118710110010110112.0,EAST 86 STREET,1 AVENUE,2 AVENUE,MANHATTAN,06/11/2019 02:52:08 PM,4.0,New,STREET OPENING PERMIT
252,M012019163A41,135430114510115712.0,WEST 83 STREET,CENTRAL PARK WEST,COLUMBUS AVENUE,MANHATTAN,06/13/2019 09:38:16 AM,4.0,Emergency,STREET OPENING PERMIT


In [ ]:
permits_gas.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 193970 entries, 107 to 1131794
Data columns (total 10 columns):
 #   Column                    Non-Null Count   Dtype  
---  ------                    --------------   -----  
 0   PermitNumber              193970 non-null  object 
 1   OFTCode                   176686 non-null  object 
 2   OnStreetName              193970 non-null  object 
 3   FromStreetName            193967 non-null  object 
 4   ToStreetName              169456 non-null  object 
 5   BoroughName               193970 non-null  object 
 6   PermitIssueDate           193970 non-null  object 
 7   PermitLinearFeet          193857 non-null  float64
 8   ApplicationTypeShortDesc  193970 non-null  object 
 9   PermitSeriesShortDesc     193970 non-null  object 
dtypes: float64(1), object(9)
memory usage: 16.3+ MB


### Merging with Lion Shapefile
- We can get almost 100% of the locations by considering only From & On Street. **Ignore the To street name from here**
- While creating the segments shapefile we can consider using **Google Maps API**

In [ ]:
streets = gpd.read_file("Streets.shp")
streets.head()

,OBJECTID,Street,SAFStreetN,FeatureTyp,SegmentTyp,IncExFlag,RB_Layer,NonPed,TrafDir,TrafSrc,SpecAddr,FaceCode,SeqNum,StreetCode,SAFStreetC,LGC1,LGC2,LGC3,LGC4,LGC5,LGC6,LGC7,LGC8,LGC9,BOE_LGC,SegmentID,SegCount,LocStatus,LZip,RZip,LBoro,RBoro,L_CD,R_CD,LATOMICPOL,RATOMICPOL,LCT2020,LCT2020Suf,RCT2020,RCT2020Suf,LCB2020,LCB2020Suf,RCB2020,RCB2020Suf,LCT2010,LCT2010Suf,RCT2010,RCT2010Suf,LCB2010,LCB2010Suf,RCB2010,RCB2010Suf,LCT2000,LCT2000Suf,RCT2000,RCT2000Suf,LCB2000,LCB2000Suf,RCB2000,RCB2000Suf,LCT1990,LCT1990Suf,RCT1990,RCT1990Suf,LAssmDist,LElectDist,RAssmDist,RElectDist,SplitElect,LSchlDist,RSchlDist,SplitSchl,LSubSect,RSubSect,SanDistInd,MapFrom,MapTo,BoroBndry,MH_RI_Flag,XFrom,YFrom,XTo,YTo,ArcCenterX,ArcCenterY,CurveFlag,Radius,NodeIDFrom,NodeIDTo,NodeLevelF,NodeLevelT,ConParity,Twisted,RW_TYPE,PhysicalID,GenericID,NYPDID,FDNYID,LBlockFace,RBlockFace,LegacyID,Status,StreetWidt,StreetWi_1,StreetWi_2,BikeLane,BIKE_TRAFD,ACTIVE_FLA,POSTED_SPE,Snow_Prior,Number_Tra,Number_Par,Number_Tot,Carto_Disp,FCC,ROW_Type,LLo_Hyphen,LHi_Hyphen,RLo_Hyphen,RHi_Hyphen,FromLeft,ToLeft,FromRight,ToRight,Join_ID,L_PD_Servi,R_PD_Servi,TRUCK_ROUT,SHAPE_Leng,geometry
0,1,EAST 168 STREET,None,0,U,None,B,None,T,DOT,None,2510,03070,226700,None,01,None,None,None,None,None,None,None,None,1,0078126,1,X,10456,10456,2,2,203,203,402,101,149,None,185,02,3001,None,1000,None,149,None,185,None,3001,None,2000,None,149,None,137,None,4000,None,1000,None,149,None,137,None,79,034,79,034,None,09,09,None,1B,1B,None,3D,3D,None,None,1010964,241812,1011265,241555,0,0,None,0,0047740,9045677,M,M,None,None,1,35231,30694,None,None,1422600653,1422602017,0078126,2,34.0,34.0,None,None,None,None,25,S,2,2,4,None,None,None,599,699,596,716,599,699,596,716,2251001000000,None,None,None,396.030947,"LINESTRING (1010964.447 241812.261, 1011265.49..."
1,2,WEST 192 STREET,None,0,U,None,B,None,A,DOT,None,7984,00040,274810,None,01,None,None,None,None,None,None,None,None,1,0079796,1,None,10468,10468,2,2,207,207,302,104,265,None,265,None,2000,None,1004,None,265,None,265,None,2000,None,1004,None,265,None,265,None,3001,None,1003,None,265,None,265,None,78,035,78,037,None,10,10,None,1A,1A,None,3C,3C,None,None,1011577,255024,1011335,255164,0,0,None,0,0048679,0048678,M,M,None,None,1,35248,30711,None,None,1522607129,1522607721,0079796,2,30.0,30.0,None,None,None,None,25,S,1,2,3,None,None,None,58,98,63,99,58,98,63,99,2798401000000,None,None,None,279.360514,"LINESTRING (1011576.687 255023.583, 1011335.15..."
2,3,UNION AVENUE,None,0,U,None,B,None,W,DOT,None,7280,00130,270420,None,01,None,None,None,None,None,None,None,None,1,0077356,4,X,10459,10459,2,2,203,203,402,401,135,None,131,None,2000,None,2003,None,135,None,131,None,2000,None,3006,None,135,None,131,None,4000,None,4001,None,135,None,131,None,79,038,79,032,None,12,12,None,1A,1A,None,6C,6C,None,None,1011601,239640,1011786,240230,0,0,None,0,0047288,0047822,M,M,None,None,1,35252,30715,None,None,1422603726,1422604132,0077356,2,34.0,34.0,None,None,None,None,25,S,1,2,3,None,None,None,1017,1079,1016,1084,1017,1079,1016,1084,2728001000000,None,None,None,618.327133,"LINESTRING (1011600.676 239639.743, 1011785.79..."
3,4,UNION AVENUE,BEHAGEN PLAYGROUND COMFORT STA,0,U,None,B,None,W,DOT,X,7280,00130,270420,212795,01,None,None,None,None,None,None,None,None,1,0077356,4,X,10459,10459,2,2,203,203,402,401,135,None,131,None,2000,None,2003,None,135,None,131,None,2000,None,3006,None,135,None,131,None,4000,None,4001,None,135,None,131,None,79,038,79,032,None,12,12,None,1A,1A,None,6C,6C,None,None,1011601,239640,1011786,240230,0,0,None,0,0047288,0047822,M,M,None,None,1,35252,30715,None,None,1422603726,1422604132,0077356,2,34.0,34.0,None,None,None,None,25,S,1,2,3,None,None,None,None,None,None,None,0,0,0,0,21279502000000X,None,None,None,618.327133,"LINESTRING (1011600.676 239639.743, 1011785.79..."
4,5,UNION AVENUE,BEHAGEN PLAYGROUND FIELD NORTH,0,U,None,B,None,W,DOT,X,7280,00130,270420,212795,01,None,None,None,None,None,None,None,None,1,0077356,4,X,10459,10459,2,2,20

In [ ]:
streets.shape

(238694, 130)

In [ ]:
streets = streets[["Street","StreetCode","geometry"]]

In [ ]:
streets_new = streets.copy()

In [ ]:
streets_new.head()

,Street,StreetCode,geometry
0,EAST 168 STREET,226700,"LINESTRING (1010964.447 241812.261, 1011265.49..."
1,WEST 192 STREET,274810,"LINESTRING (1011576.687 255023.583, 1011335.15..."
2,UNION AVENUE,270420,"LINESTRING (1011600.676 239639.743, 1011785.79..."
3,UNION AVENUE,270420,"LINESTRING (1011600.676 239639.743, 1011785.79..."
4,UNION AVENUE,270420,"LINESTRING (1011600.676 239639.743, 1011785.79..."


In [ ]:
streets_new.iloc[3,:] == streets_new.iloc[4,:]

Street        True
StreetCode    True
geometry      True
dtype: bool

In [ ]:
#combining street segments
from shapely.ops import unary_union

grouped_gdf = streets_new.groupby('StreetCode')

# Create an empty GeoDataFrame to hold the unioned geometries
unioned_gdf = gpd.GeoDataFrame()

# Iterate over the groups and perform the union operation
for group_name, group_data in grouped_gdf:
    # Get the unioned geometry for this group
    unioned_geometry = unary_union(group_data.geometry)
    
    # Create a GeoDataFrame for this group with the unioned geometry
    group_gdf = gpd.GeoDataFrame({'StreetCode': [group_name], 'geometry': [unioned_geometry]})
    
    # Append the group GeoDataFrame to the overall unioned GeoDataFrame
    unioned_gdf = unioned_gdf.append(group_gdf, ignore_index=True)

In [ ]:
unioned_gdf.head()

,StreetCode,geometry
0,100001,"MULTILINESTRING ((973038.594 194089.284, 97303..."
1,100008,"MULTILINESTRING ((982474.321 206347.963, 98216..."
2,100009,"MULTILINESTRING ((985234.586 219027.174, 98506..."
3,100013,"MULTILINESTRING ((1005621.313 228030.268, 1005..."
4,100016,"MULTILINESTRING ((980136.018 199662.670, 98005..."


In [ ]:
# convert to integers using pd.to_numeric()
permits_gas['OFTCode'] = pd.to_numeric(permits_gas['OFTCode'], errors='coerce').astype('Int64')

# convert to strings and split into 3 columns
permits_gas['OFTCode'] = permits_gas['OFTCode'].astype(str)
permits_gas['On_Code'] = permits_gas['OFTCode'].str[:6]
permits_gas['From_Code'] = permits_gas['OFTCode'].str[6:12]

In [ ]:
permits_gas.head()

,PermitNumber,OFTCode,OnStreetName,FromStreetName,ToStreetName,BoroughName,PermitIssueDate,PermitLinearFeet,ApplicationTypeShortDesc,PermitSeriesShortDesc,On_Code,From_Code
107,B012023089A25,<NA>,AVENUE M,EAST 35 STREET,NaN,BROOKLYN,03/30/2023 06:28:03 AM,75.0,New,STREET OPENING PERMIT,<NA>,
238,M012019112A85,110210118630118656,3 AVENUE,EAST 82 STREET,EAST 83 STREET,MANHATTAN,04/22/2019 09:59:18 AM,4.0,New,STREET OPENING PERMIT,110210,118630
242,M012019121A35,110010118310118336,1 AVENUE,EAST 66 STREET,EAST 67 STREET,MANHATTAN,05/01/2019 10:12:46 AM,25.0,New,STREET OPENING PERMIT,110010,118310
251,M012019162B93,118710110010110112,EAST 86 STREET,1 AVENUE,2 AVENUE,MANHATTAN,06/11/2019 02:52:08 PM,4.0,New,STREET OPENING PERMIT,118710,110010
252,M012019163A41,135430114510115712,WEST 83 STREET,CENTRAL PARK WEST,COLUMBUS AVENUE,MANHATTAN,06/13/2019 09:38:16 AM,4.0,Emergency,STREET OPENING PERMIT,135430,114510


In [ ]:
permits_gas = permits_gas.dropna(subset=['On_Code', 'From_Code'])
permits_gas.shape

(193970, 12)

In [ ]:
permits_gas.head()

,PermitNumber,OFTCode,OnStreetName,FromStreetName,ToStreetName,BoroughName,PermitIssueDate,PermitLinearFeet,ApplicationTypeShortDesc,PermitSeriesShortDesc,On_Code,From_Code
107,B012023089A25,<NA>,AVENUE M,EAST 35 STREET,NaN,BROOKLYN,03/30/2023 06:28:03 AM,75.0,New,STREET OPENING PERMIT,<NA>,
238,M012019112A85,110210118630118656,3 AVENUE,EAST 82 STREET,EAST 83 STREET,MANHATTAN,04/22/2019 09:59:18 AM,4.0,New,STREET OPENING PERMIT,110210,118630
242,M012019121A35,110010118310118336,1 AVENUE,EAST 66 STREET,EAST 67 STREET,MANHATTAN,05/01/2019 10:12:46 AM,25.0,New,STREET OPENING PERMIT,110010,118310
251,M012019162B93,118710110010110112,EAST 86 STREET,1 AVENUE,2 AVENUE,MANHATTAN,06/11/2019 02:52:08 PM,4.0,New,STREET OPENING PERMIT,118710,110010
252,M012019163A41,135430114510115712,WEST 83 STREET,CENTRAL PARK WEST,COLUMBUS AVENUE,MANHATTAN,06/13/2019 09:38:16 AM,4.0,Emergency,STREET OPENING PERMIT,135430,114510


In [ ]:
permits_gas = permits_gas[permits_gas["On_Code"]!="<NA>"]
permits_gas.head()

,PermitNumber,OFTCode,OnStreetName,FromStreetName,ToStreetName,BoroughName,PermitIssueDate,PermitLinearFeet,ApplicationTypeShortDesc,PermitSeriesShortDesc,On_Code,From_Code
238,M012019112A85,110210118630118656,3 AVENUE,EAST 82 STREET,EAST 83 STREET,MANHATTAN,04/22/2019 09:59:18 AM,4.0,New,STREET OPENING PERMIT,110210,118630
242,M012019121A35,110010118310118336,1 AVENUE,EAST 66 STREET,EAST 67 STREET,MANHATTAN,05/01/2019 10:12:46 AM,25.0,New,STREET OPENING PERMIT,110010,118310
251,M012019162B93,118710110010110112,EAST 86 STREET,1 AVENUE,2 AVENUE,MANHATTAN,06/11/2019 02:52:08 PM,4.0,New,STREET OPENING PERMIT,118710,110010
252,M012019163A41,135430114510115712,WEST 83 STREET,CENTRAL PARK WEST,COLUMBUS AVENUE,MANHATTAN,06/13/2019 09:38:16 AM,4.0,Emergency,STREET OPENING PERMIT,135430,114510
257,M012019225A84,118230110010197664,EAST 62 STREET,1 AVENUE,QUEENSBORO BRDG EXIT,MANHATTAN,08/13/2019 09:27:06 AM,4.0,New,STREET OPENING PERMIT,118230,110010


#### Intersection of From and On Geometry

In [ ]:
on_df = pd.merge(permits_gas, unioned_gdf, how='inner', left_on='On_Code', right_on='StreetCode')
on_df = on_df.rename(columns={'geometry': 'On_Geometry'})
on_df.shape

(176604, 14)

In [ ]:
from_df = pd.merge(permits_gas, unioned_gdf, how='inner', left_on='From_Code', right_on='StreetCode')
from_df = from_df.rename(columns={'geometry': 'From_Geometry'})
from_df.shape

(172282, 14)

In [ ]:
comb = pd.merge(on_df[["PermitNumber","On_Geometry","PermitLinearFeet"]],from_df[["PermitNumber","From_Geometry"]],on="PermitNumber")
comb.head()

,PermitNumber,On_Geometry,PermitLinearFeet,From_Geometry
0,M012019112A85,"MULTILINESTRING ((989928.328 210272.873, 99005...",4.0,"MULTILINESTRING ((995133.997 222824.613, 99554..."
1,M012018303A88,"MULTILINESTRING ((989928.328 210272.873, 99005...",25.0,"MULTILINESTRING ((997384.330 223729.788, 99800..."
2,M012018303A89,"MULTILINESTRING ((989928.328 210272.873, 99005...",25.0,"MULTILINESTRING ((996395.254 223977.839, 99681..."
3,M012018303B26,"MULTILINESTRING ((989928.328 210272.873, 99005...",40.0,"MULTILINESTRING ((997384.330 223729.788, 99800..."
4,M012018331A53,"MULTILINESTRING ((989928.328 210272.873, 99005...",240.0,"MULTILINESTRING ((992856.164 215558.335, 99347..."


In [ ]:
comb.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 172202 entries, 0 to 172201
Data columns (total 4 columns):
 #   Column            Non-Null Count   Dtype   
---  ------            --------------   -----   
 0   PermitNumber      172202 non-null  object  
 1   On_Geometry       172202 non-null  geometry
 2   PermitLinearFeet  172166 non-null  float64 
 3   From_Geometry     172202 non-null  geometry
dtypes: float64(1), geometry(2), object(1)
memory usage: 6.6+ MB


In [ ]:
comb.nunique()

PermitNumber        172202
On_Geometry           5613
PermitLinearFeet       722
From_Geometry         5699
dtype: int64

#### Getting point coordinate of intersection of on and from street

In [ ]:
import pandas as pd
import geopandas as gpd
from shapely.wkt import loads

In [ ]:
on_gdf = gpd.GeoDataFrame(
    on_df[["PermitNumber","On_Geometry","PermitLinearFeet"]], 
    crs='EPSG:4326', 
    geometry=on_df['On_Geometry']
)

In [ ]:
from_gdf = gpd.GeoDataFrame(
    from_df[["PermitNumber","From_Geometry","PermitLinearFeet"]], 
    crs='EPSG:4326', 
    geometry=from_df['From_Geometry']
)

In [ ]:
on_gdf.crs

<Geographic 2D CRS: EPSG:4326>
Name: WGS 84
Axis Info [ellipsoidal]:
- Lat[north]: Geodetic latitude (degree)
- Lon[east]: Geodetic longitude (degree)
Area of Use:
- name: World.
- bounds: (-180.0, -90.0, 180.0, 90.0)
Datum: World Geodetic System 1984 ensemble
- Ellipsoid: WGS 84
- Prime Meridian: Greenwich

In [ ]:
merged_gdf = on_gdf.merge(from_gdf, on='PermitNumber')
merged_gdf.head()

,PermitNumber,On_Geometry,PermitLinearFeet_x,geometry_x,From_Geometry,PermitLinearFeet_y,geometry_y
0,M012019112A85,"MULTILINESTRING ((989928.328 210272.873, 99005...",4.0,"MULTILINESTRING ((989928.32848 210272.87341, 9...","MULTILINESTRING ((995133.997 222824.613, 99554...",4.0,"MULTILINESTRING ((995133.99703 222824.61300, 9..."
1,M012018303A88,"MULTILINESTRING ((989928.328 210272.873, 99005...",25.0,"MULTILINESTRING ((989928.32848 210272.87341, 9...","MULTILINESTRING ((997384.330 223729.788, 99800...",25.0,"MULTILINESTRING ((997384.33033 223729.78834, 9..."
2,M012018303A89,"MULTILINESTRING ((989928.328 210272.873, 99005...",25.0,"MULTILINESTRING ((989928.32848 210272.87341, 9...","MULTILINESTRING ((996395.254 223977.839, 99681...",25.0,"MULTILINESTRING ((996395.25419 223977.83896, 9..."
3,M012018303B26,"MULTILINESTRING ((989928.328 210272.873, 99005...",40.0,"MULTILINESTRING ((989928.32848 210272.87341, 9...","MULTILINESTRING ((997384.330 223729.788, 99800...",40.0,"MULTILINESTRING ((997384.33033 223729.78834, 9..."
4,M012018331A53,"MULTILINESTRING ((989928.328 210272.873, 99005...",240.0,"MULTILINESTRING ((989928.32848 210272.87341, 9...","MULTILINESTRING ((992856.164 215558.335, 99347...",240.0,"MULTILINESTRING ((992856.16362 215558.33516, 9..."


In [ ]:
merged_gdf.shape

(172202, 7)

In [ ]:
merged_gdf["PermitNumber"].nunique()

172202

In [ ]:
for index, row in merged_gdf.iterrows():
    try:
        # find intersection of From_Geometry with On_Geometry
        intersect = row['From_Geometry'].intersection(row['On_Geometry'])
        merged_gdf.loc[index, 'From_On'] = intersect
    except:
        # if an error occurs, drop the row from the dataframe
        merged_gdf.drop(index, inplace=True)

In [ ]:
merged_gdf.head()

,PermitNumber,On_Geometry,PermitLinearFeet_x,geometry_x,From_Geometry,PermitLinearFeet_y,geometry_y,From_On
0,M012019112A85,"MULTILINESTRING ((989928.328 210272.873, 99005...",4.0,"MULTILINESTRING ((989928.32848 210272.87341, 9...","MULTILINESTRING ((995133.997 222824.613, 99554...",4.0,"MULTILINESTRING ((995133.99703 222824.61300, 9...",POINT (996472.5933904499 222083.68606638908)
1,M012018303A88,"MULTILINESTRING ((989928.328 210272.873, 99005...",25.0,"MULTILINESTRING ((989928.32848 210272.87341, 9...","MULTILINESTRING ((997384.330 223729.788, 99800...",25.0,"MULTILINESTRING ((997384.33033 223729.78834, 9...",POINT (997384.3303337544 223729.78834457695)
2,M012018303A89,"MULTILINESTRING ((989928.328 210272.873, 99005...",25.0,"MULTILINESTRING ((989928.32848 210272.87341, 9...","MULTILINESTRING ((996395.254 223977.839, 99681...",25.0,"MULTILINESTRING ((996395.25419 223977.83896, 9...",POINT (997257.2078277171 223501.07163371146)
3,M012018303B26,"MULTILINESTRING ((989928.328 210272.873, 99005...",40.0,"MULTILINESTRING ((989928.32848 210272.87341, 9...","MULTILINESTRING ((997384.330 223729.788, 99800...",40.0,"MULTILINESTRING ((997384.33033 223729.78834, 9...",POINT (997384.3303337544 223729.78834457695)
4,M012018331A53,"MULTILINESTRING ((989928.328 210272.873, 99005...",240.0,"MULTILINESTRING ((989928.32848 210272.87341, 9...","MULTILINESTRING ((992856.164 215558.335, 99347...",240.0,"MULTILINESTRING ((992856.16362 215558.33516, 9...",POINT (992856.1636186838 215558.33515645564)


#### Bring in other information from permits data

In [ ]:
permits_gas.head()

,PermitNumber,OFTCode,OnStreetName,FromStreetName,ToStreetName,BoroughName,PermitIssueDate,PermitLinearFeet,ApplicationTypeShortDesc,PermitSeriesShortDesc,On_Code,From_Code
238,M012019112A85,110210118630118656,3 AVENUE,EAST 82 STREET,EAST 83 STREET,MANHATTAN,04/22/2019 09:59:18 AM,4.0,New,STREET OPENING PERMIT,110210,118630
242,M012019121A35,110010118310118336,1 AVENUE,EAST 66 STREET,EAST 67 STREET,MANHATTAN,05/01/2019 10:12:46 AM,25.0,New,STREET OPENING PERMIT,110010,118310
251,M012019162B93,118710110010110112,EAST 86 STREET,1 AVENUE,2 AVENUE,MANHATTAN,06/11/2019 02:52:08 PM,4.0,New,STREET OPENING PERMIT,118710,110010
252,M012019163A41,135430114510115712,WEST 83 STREET,CENTRAL PARK WEST,COLUMBUS AVENUE,MANHATTAN,06/13/2019 09:38:16 AM,4.0,Emergency,STREET OPENING PERMIT,135430,114510
257,M012019225A84,118230110010197664,EAST 62 STREET,1 AVENUE,QUEENSBORO BRDG EXIT,MANHATTAN,08/13/2019 09:27:06 AM,4.0,New,STREET OPENING PERMIT,118230,110010


In [ ]:
permits_points = pd.merge(permits_gas,merged_gdf[["PermitNumber","From_On"]],on="PermitNumber")
permits_points.head()

,PermitNumber,OFTCode,OnStreetName,FromStreetName,ToStreetName,BoroughName,PermitIssueDate,PermitLinearFeet,ApplicationTypeShortDesc,PermitSeriesShortDesc,On_Code,From_Code,From_On
0,M012019112A85,110210118630118656,3 AVENUE,EAST 82 STREET,EAST 83 STREET,MANHATTAN,04/22/2019 09:59:18 AM,4.0,New,STREET OPENING PERMIT,110210,118630,POINT (996472.5933904499 222083.68606638908)
1,M012019121A35,110010118310118336,1 AVENUE,EAST 66 STREET,EAST 67 STREET,MANHATTAN,05/01/2019 10:12:46 AM,25.0,New,STREET OPENING PERMIT,110010,118310,POINT (995668.3200522512 217621.59205445647)
2,M012019162B93,118710110010110112,EAST 86 STREET,1 AVENUE,2 AVENUE,MANHATTAN,06/11/2019 02:52:08 PM,4.0,New,STREET OPENING PERMIT,118710,110010,POINT (998271.3237758875 222317.78757750988)
3,M012019163A41,135430114510115712,WEST 83 STREET,CENTRAL PARK WEST,COLUMBUS AVENUE,MANHATTAN,06/13/2019 09:38:16 AM,4.0,Emergency,STREET OPENING PERMIT,135430,114510,POINT (992347.5195945203 224670.15378925204)
4,M012019225A84,118230110010197664,EAST 62 STREET,1 AVENUE,QUEENSBORO BRDG EXIT,MANHATTAN,08/13/2019 09:27:06 AM,4.0,New,STREET OPENING PERMIT,118230,110010,POINT (995164.7120283246 216708.64701110125)


In [ ]:
permits_points.shape

(172202, 13)

In [ ]:
#Get Date
permits_points['Date'] = pd.to_datetime(permits_points['PermitIssueDate'])

# Extract the date portion
permits_points['Date'] = permits_points['Date'].dt.date

permits_points.head()

,PermitNumber,OFTCode,OnStreetName,FromStreetName,ToStreetName,BoroughName,PermitIssueDate,PermitLinearFeet,ApplicationTypeShortDesc,PermitSeriesShortDesc,On_Code,From_Code,From_On,Date
0,M012019112A85,110210118630118656,3 AVENUE,EAST 82 STREET,EAST 83 STREET,MANHATTAN,04/22/2019 09:59:18 AM,4.0,New,STREET OPENING PERMIT,110210,118630,POINT (996472.5933904499 222083.68606638908),2019-04-22
1,M012019121A35,110010118310118336,1 AVENUE,EAST 66 STREET,EAST 67 STREET,MANHATTAN,05/01/2019 10:12:46 AM,25.0,New,STREET OPENING PERMIT,110010,118310,POINT (995668.3200522512 217621.59205445647),2019-05-01
2,M012019162B93,118710110010110112,EAST 86 STREET,1 AVENUE,2 AVENUE,MANHATTAN,06/11/2019 02:52:08 PM,4.0,New,STREET OPENING PERMIT,118710,110010,POINT (998271.3237758875 222317.78757750988),2019-06-11
3,M012019163A41,135430114510115712,WEST 83 STREET,CENTRAL PARK WEST,COLUMBUS AVENUE,MANHATTAN,06/13/2019 09:38:16 AM,4.0,Emergency,STREET OPENING PERMIT,135430,114510,POINT (992347.5195945203 224670.15378925204),2019-06-13
4,M012019225A84,118230110010197664,EAST 62 STREET,1 AVENUE,QUEENSBORO BRDG EXIT,MANHATTAN,08/13/2019 09:27:06 AM,4.0,New,STREET OPENING PERMIT,118230,110010,POINT (995164.7120283246 216708.64701110125),2019-08-13


In [ ]:
permits_points.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 172202 entries, 0 to 172201
Data columns (total 14 columns):
 #   Column                    Non-Null Count   Dtype  
---  ------                    --------------   -----  
 0   PermitNumber              172202 non-null  object 
 1   OFTCode                   172202 non-null  object 
 2   OnStreetName              172202 non-null  object 
 3   FromStreetName            172199 non-null  object 
 4   ToStreetName              150647 non-null  object 
 5   BoroughName               172202 non-null  object 
 6   PermitIssueDate           172202 non-null  object 
 7   PermitLinearFeet          172166 non-null  float64
 8   ApplicationTypeShortDesc  172202 non-null  object 
 9   PermitSeriesShortDesc     172202 non-null  object 
 10  On_Code                   172202 non-null  object 
 11  From_Code                 172202 non-null  object 
 12  From_On                   172202 non-null  object 
 13  Date                      172202 non-null  o

In [ ]:
#Get Year, Month
permits_points['Date'] = pd.to_datetime(permits_points['Date'])
permits_points['Year'] = permits_points['Date'].dt.year
permits_points['Month'] = permits_points['Date'].dt.month
permits_points.head()

,PermitNumber,OFTCode,OnStreetName,FromStreetName,ToStreetName,BoroughName,PermitIssueDate,PermitLinearFeet,ApplicationTypeShortDesc,PermitSeriesShortDesc,On_Code,From_Code,From_On,Date,Year,Month
0,M012019112A85,110210118630118656,3 AVENUE,EAST 82 STREET,EAST 83 STREET,MANHATTAN,04/22/2019 09:59:18 AM,4.0,New,STREET OPENING PERMIT,110210,118630,POINT (996472.5933904499 222083.68606638908),2019-04-22,2019,4
1,M012019121A35,110010118310118336,1 AVENUE,EAST 66 STREET,EAST 67 STREET,MANHATTAN,05/01/2019 10:12:46 AM,25.0,New,STREET OPENING PERMIT,110010,118310,POINT (995668.3200522512 217621.59205445647),2019-05-01,2019,5
2,M012019162B93,118710110010110112,EAST 86 STREET,1 AVENUE,2 AVENUE,MANHATTAN,06/11/2019 02:52:08 PM,4.0,New,STREET OPENING PERMIT,118710,110010,POINT (998271.3237758875 222317.78757750988),2019-06-11,2019,6
3,M012019163A41,135430114510115712,WEST 83 STREET,CENTRAL PARK WEST,COLUMBUS AVENUE,MANHATTAN,06/13/2019 09:38:16 AM,4.0,Emergency,STREET OPENING PERMIT,135430,114510,POINT (992347.5195945203 224670.15378925204),2019-06-13,2019,6
4,M012019225A84,118230110010197664,EAST 62 STREET,1 AVENUE,QUEENSBORO BRDG EXIT,MANHATTAN,08/13/2019 09:27:06 AM,4.0,New,STREET OPENING PERMIT,118230,110010,POINT (995164.7120283246 216708.64701110125),2019-08-13,2019,8


In [ ]:
permits_points['Year'].value_counts()

2019    95783
2022    46477
2018    14580
2020     8585
2021     5756
2017      986
2023       21
2015        7
2016        7
Name: Year, dtype: int64

### Block Group

In [ ]:
bg = gpd.read_file("G:/Independent Study/Block Group/tl_2017_36_bg.shp")
bg.head()

,STATEFP,COUNTYFP,TRACTCE,BLKGRPCE,GEOID,NAMELSAD,MTFCC,FUNCSTAT,ALAND,AWATER,INTPTLAT,INTPTLON,geometry
0,36,005,017500,2,360050175002,Block Group 2,G5030,S,23694,0,+40.8296335,-073.9152037,"POLYGON ((-73.91570 40.83054, -73.91485 40.830..."
1,36,005,014100,1,360050141001,Block Group 1,G5030,S,55934,0,+40.8245444,-073.9131209,"POLYGON ((-73.91661 40.82499, -73.91592 40.825..."
2,36,005,014500,1,360050145001,Block Group 1,G5030,S,47026,0,+40.8309816,-073.9050870,"POLYGON ((-73.90584 40.83106, -73.90505 40.832..."
3,36,005,007500,2,360050075002,Block Group 2,G5030,S,109058,0,+40.8202395,-073.9080678,"POLYGON ((-73.91035 40.81995, -73.91022 40.820..."
4,36,005,041800,1,360050418001,Block Group 1,G5030,S,140793,0,+40.8989195,-073.8564428,"POLYGON ((-73.86288 40.89515, -73.86146 40.897..."


In [ ]:
bg.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 15463 entries, 0 to 15462
Data columns (total 13 columns):
 #   Column    Non-Null Count  Dtype   
---  ------    --------------  -----   
 0   STATEFP   15463 non-null  object  
 1   COUNTYFP  15463 non-null  object  
 2   TRACTCE   15463 non-null  object  
 3   BLKGRPCE  15463 non-null  object  
 4   GEOID     15463 non-null  object  
 5   NAMELSAD  15463 non-null  object  
 6   MTFCC     15463 non-null  object  
 7   FUNCSTAT  15463 non-null  object  
 8   ALAND     15463 non-null  int64   
 9   AWATER    15463 non-null  int64   
 10  INTPTLAT  15463 non-null  object  
 11  INTPTLON  15463 non-null  object  
 12  geometry  15463 non-null  geometry
dtypes: geometry(1), int64(2), object(10)
memory usage: 1.5+ MB


In [ ]:
bg = bg[(bg["COUNTYFP"]=="005") | (bg["COUNTYFP"]=="047") | (bg["COUNTYFP"]=="061") | (bg["COUNTYFP"]=="081") | (bg["COUNTYFP"]=="085")]
bg.head()

,STATEFP,COUNTYFP,TRACTCE,BLKGRPCE,GEOID,NAMELSAD,MTFCC,FUNCSTAT,ALAND,AWATER,INTPTLAT,INTPTLON,geometry
0,36,005,017500,2,360050175002,Block Group 2,G5030,S,23694,0,+40.8296335,-073.9152037,"POLYGON ((-73.91570 40.83054, -73.91485 40.830..."
1,36,005,014100,1,360050141001,Block Group 1,G5030,S,55934,0,+40.8245444,-073.9131209,"POLYGON ((-73.91661 40.82499, -73.91592 40.825..."
2,36,005,014500,1,360050145001,Block Group 1,G5030,S,47026,0,+40.8309816,-073.9050870,"POLYGON ((-73.90584 40.83106, -73.90505 40.832..."
3,36,005,007500,2,360050075002,Block Group 2,G5030,S,109058,0,+40.8202395,-073.9080678,"POLYGON ((-73.91035 40.81995, -73.91022 40.820..."
4,36,005,041800,1,360050418001,Block Group 1,G5030,S,140793,0,+40.8989195,-073.8564428,"POLYGON ((-73.86288 40.89515, -73.86146 40.897..."


In [ ]:
bg["COUNTYFP"].value_counts()

047    2085
081    1746
061    1170
005    1154
085     338
Name: COUNTYFP, dtype: int64

In [ ]:
target_crs = 'EPSG:2263'

# Convert to the target coordinate system
bg_nyc = bg.to_crs(target_crs)

In [ ]:
bg_nyc.crs

<Derived Projected CRS: EPSG:2263>
Name: NAD83 / New York Long Island (ftUS)
Axis Info [cartesian]:
- X[east]: Easting (US survey foot)
- Y[north]: Northing (US survey foot)
Area of Use:
- name: United States (USA) - New York - counties of Bronx; Kings; Nassau; New York; Queens; Richmond; Suffolk.
- bounds: (-74.26, 40.47, -71.8, 41.3)
Coordinate Operation:
- name: SPCS83 New York Long Island zone (US Survey feet)
- method: Lambert Conic Conformal (2SP)
Datum: North American Datum 1983
- Ellipsoid: GRS 1980
- Prime Meridian: Greenwich

In [ ]:
#bg_nyc.to_file("bg_nyc.shp")

#### Block Group for Each Permit

In [ ]:
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point

In [ ]:
df = permits_points.copy()

df.head()

,PermitNumber,OFTCode,OnStreetName,FromStreetName,ToStreetName,BoroughName,PermitIssueDate,PermitLinearFeet,ApplicationTypeShortDesc,PermitSeriesShortDesc,On_Code,From_Code,From_On,Date,Year,Month
0,M012019112A85,110210118630118656,3 AVENUE,EAST 82 STREET,EAST 83 STREET,MANHATTAN,04/22/2019 09:59:18 AM,4.0,New,STREET OPENING PERMIT,110210,118630,POINT (996472.5933904499 222083.68606638908),2019-04-22,2019,4
1,M012019121A35,110010118310118336,1 AVENUE,EAST 66 STREET,EAST 67 STREET,MANHATTAN,05/01/2019 10:12:46 AM,25.0,New,STREET OPENING PERMIT,110010,118310,POINT (995668.3200522512 217621.59205445647),2019-05-01,2019,5
2,M012019162B93,118710110010110112,EAST 86 STREET,1 AVENUE,2 AVENUE,MANHATTAN,06/11/2019 02:52:08 PM,4.0,New,STREET OPENING PERMIT,118710,110010,POINT (998271.3237758875 222317.78757750988),2019-06-11,2019,6
3,M012019163A41,135430114510115712,WEST 83 STREET,CENTRAL PARK WEST,COLUMBUS AVENUE,MANHATTAN,06/13/2019 09:38:16 AM,4.0,Emergency,STREET OPENING PERMIT,135430,114510,POINT (992347.5195945203 224670.15378925204),2019-06-13,2019,6
4,M012019225A84,118230110010197664,EAST 62 STREET,1 AVENUE,QUEENSBORO BRDG EXIT,MANHATTAN,08/13/2019 09:27:06 AM,4.0,New,STREET OPENING PERMIT,118230,110010,POINT (995164.7120283246 216708.64701110125),2019-08-13,2019,8


In [ ]:
gdf = gpd.GeoDataFrame(df, geometry='From_On')
gdf.head()

,PermitNumber,OFTCode,OnStreetName,FromStreetName,ToStreetName,BoroughName,PermitIssueDate,PermitLinearFeet,ApplicationTypeShortDesc,PermitSeriesShortDesc,On_Code,From_Code,From_On,Date,Year,Month
0,M012019112A85,110210118630118656,3 AVENUE,EAST 82 STREET,EAST 83 STREET,MANHATTAN,04/22/2019 09:59:18 AM,4.0,New,STREET OPENING PERMIT,110210,118630,POINT (996472.593 222083.686),2019-04-22,2019,4
1,M012019121A35,110010118310118336,1 AVENUE,EAST 66 STREET,EAST 67 STREET,MANHATTAN,05/01/2019 10:12:46 AM,25.0,New,STREET OPENING PERMIT,110010,118310,POINT (995668.320 217621.592),2019-05-01,2019,5
2,M012019162B93,118710110010110112,EAST 86 STREET,1 AVENUE,2 AVENUE,MANHATTAN,06/11/2019 02:52:08 PM,4.0,New,STREET OPENING PERMIT,118710,110010,POINT (998271.324 222317.788),2019-06-11,2019,6
3,M012019163A41,135430114510115712,WEST 83 STREET,CENTRAL PARK WEST,COLUMBUS AVENUE,MANHATTAN,06/13/2019 09:38:16 AM,4.0,Emergency,STREET OPENING PERMIT,135430,114510,POINT (992347.520 224670.154),2019-06-13,2019,6
4,M012019225A84,118230110010197664,EAST 62 STREET,1 AVENUE,QUEENSBORO BRDG EXIT,MANHATTAN,08/13/2019 09:27:06 AM,4.0,New,STREET OPENING PERMIT,118230,110010,POINT (995164.712 216708.647),2019-08-13,2019,8


In [ ]:
gdf.crs = "EPSG:2263"

In [ ]:
gdf = gdf.to_crs(bg.crs)

In [ ]:
gdf.crs

<Geographic 2D CRS: EPSG:4269>
Name: NAD83
Axis Info [ellipsoidal]:
- Lat[north]: Geodetic latitude (degree)
- Lon[east]: Geodetic longitude (degree)
Area of Use:
- name: North America - onshore and offshore: Canada - Alberta; British Columbia; Manitoba; New Brunswick; Newfoundland and Labrador; Northwest Territories; Nova Scotia; Nunavut; Ontario; Prince Edward Island; Quebec; Saskatchewan; Yukon. Puerto Rico. United States (USA) - Alabama; Alaska; Arizona; Arkansas; California; Colorado; Connecticut; Delaware; Florida; Georgia; Hawaii; Idaho; Illinois; Indiana; Iowa; Kansas; Kentucky; Louisiana; Maine; Maryland; Massachusetts; Michigan; Minnesota; Mississippi; Missouri; Montana; Nebraska; Nevada; New Hampshire; New Jersey; New Mexico; New York; North Carolina; North Dakota; Ohio; Oklahoma; Oregon; Pennsylvania; Rhode Island; South Carolina; South Dakota; Tennessee; Texas; Utah; Vermont; Virginia; Washington; West Virginia; Wisconsin; Wyoming. US Virgin Islands. British Virgin Islands

In [ ]:
bg.crs

<Geographic 2D CRS: EPSG:4269>
Name: NAD83
Axis Info [ellipsoidal]:
- Lat[north]: Geodetic latitude (degree)
- Lon[east]: Geodetic longitude (degree)
Area of Use:
- name: North America - onshore and offshore: Canada - Alberta; British Columbia; Manitoba; New Brunswick; Newfoundland and Labrador; Northwest Territories; Nova Scotia; Nunavut; Ontario; Prince Edward Island; Quebec; Saskatchewan; Yukon. Puerto Rico. United States (USA) - Alabama; Alaska; Arizona; Arkansas; California; Colorado; Connecticut; Delaware; Florida; Georgia; Hawaii; Idaho; Illinois; Indiana; Iowa; Kansas; Kentucky; Louisiana; Maine; Maryland; Massachusetts; Michigan; Minnesota; Mississippi; Missouri; Montana; Nebraska; Nevada; New Hampshire; New Jersey; New Mexico; New York; North Carolina; North Dakota; Ohio; Oklahoma; Oregon; Pennsylvania; Rhode Island; South Carolina; South Dakota; Tennessee; Texas; Utah; Vermont; Virginia; Washington; West Virginia; Wisconsin; Wyoming. US Virgin Islands. British Virgin Islands

In [ ]:
#within spatial join
permits_bg_gas = gpd.sjoin(gdf, bg[["GEOID","COUNTYFP","geometry"]], op='within')
permits_bg_gas.head()

,PermitNumber,OFTCode,OnStreetName,FromStreetName,ToStreetName,BoroughName,PermitIssueDate,PermitLinearFeet,ApplicationTypeShortDesc,PermitSeriesShortDesc,On_Code,From_Code,From_On,Date,Year,Month,index_right,GEOID,COUNTYFP
0,M012019112A85,110210118630118656,3 AVENUE,EAST 82 STREET,EAST 83 STREET,MANHATTAN,04/22/2019 09:59:18 AM,4.0,New,STREET OPENING PERMIT,110210,118630,POINT (-73.95587 40.77624),2019-04-22,2019,4,4361,360610140001,061
63890,M012017159B99,118630124890127792,EAST 82 STREET,LEXINGTON AVENUE,PARK AVENUE,MANHATTAN,06/12/2017 02:13:21 PM,100.0,Emergency,STREET OPENING PERMIT,118630,124890,POINT (-73.95747 40.77691),2017-06-12,2017,6,4361,360610140001,061
63891,M012017160D00,118630124890127792,EAST 82 STREET,LEXINGTON AVENUE,PARK AVENUE,MANHATTAN,07/26/2017 11:09:51 AM,10.0,Emergency,STREET OPENING PERMIT,118630,124890,POINT (-73.95747 40.77691),2017-07-26,2017,7,4361,360610140001,061
63905,M012017188A09,118630110210124896,EAST 82 STREET,3 AVENUE,LEXINGTON AVENUE,MANHATTAN,07/12/2017 04:41:30 PM,4.0,Emergency,STREET OPENING PERMIT,118630,110210,POINT (-73.95587 40.77624),2017-07-12,2017,7,4361,360610140001,061
63906,M012017191A66,118630110210124896,EAST 82 STREET,3 AVENUE,LEXINGTON AVENUE,MANHATTAN,07/26/2017 10:34:17 AM,4.0,Emergency,STREET OPENING PERMIT,118630,110210,POINT (-73.95587 40.77624),2017-07-26,2017,7,4361,360610140001,061


In [ ]:
permits_bg_gas["GEOID"].nunique()

5435

In [ ]:
bg["GEOID"].nunique()

6493

#### Creating Block Group geometry shapefile

In [ ]:
permits_bg_polygon_gas = pd.merge(permits_bg_gas,bg[["GEOID","COUNTYFP","geometry"]],on=["GEOID","COUNTYFP"])
permits_bg_polygon_gas.head()

,PermitNumber,OFTCode,OnStreetName,FromStreetName,ToStreetName,BoroughName,PermitIssueDate,PermitLinearFeet,ApplicationTypeShortDesc,PermitSeriesShortDesc,On_Code,From_Code,From_On,Date,Year,Month,index_right,GEOID,COUNTYFP,geometry
0,M012019112A85,110210118630118656,3 AVENUE,EAST 82 STREET,EAST 83 STREET,MANHATTAN,04/22/2019 09:59:18 AM,4.0,New,STREET OPENING PERMIT,110210,118630,POINT (-73.95587 40.77624),2019-04-22,2019,4,4361,360610140001,061,"POLYGON ((-73.95701 40.77754, -73.95655 40.778..."
1,M012017159B99,118630124890127792,EAST 82 STREET,LEXINGTON AVENUE,PARK AVENUE,MANHATTAN,06/12/2017 02:13:21 PM,100.0,Emergency,STREET OPENING PERMIT,118630,124890,POINT (-73.95747 40.77691),2017-06-12,2017,6,4361,360610140001,061,"POLYGON ((-73.95701 40.77754, -73.95655 40.778..."
2,M012017160D00,118630124890127792,EAST 82 STREET,LEXINGTON AVENUE,PARK AVENUE,MANHATTAN,07/26/2017 11:09:51 AM,10.0,Emergency,STREET OPENING PERMIT,118630,124890,POINT (-73.95747 40.77691),2017-07-26,2017,7,4361,360610140001,061,"POLYGON ((-73.95701 40.77754, -73.95655 40.778..."
3,M012017188A09,118630110210124896,EAST 82 STREET,3 AVENUE,LEXINGTON AVENUE,MANHATTAN,07/12/2017 04:41:30 PM,4.0,Emergency,STREET OPENING PERMIT,118630,110210,POINT (-73.95587 40.77624),2017-07-12,2017,7,4361,360610140001,061,"POLYGON ((-73.95701 40.77754, -73.95655 40.778..."
4,M012017191A66,118630110210124896,EAST 82 STREET,3 AVENUE,LEXINGTON AVENUE,MANHATTAN,07/26/2017 10:34:17 AM,4.0,Emergency,STREET OPENING PERMIT,118630,110210,POINT (-73.95587 40.77624),2017-07-26,2017,7,4361,360610140001,061,"POLYGON ((-73.95701 40.77754, -73.95655 40.778..."


In [ ]:
permits_bg_polygon_gas.head()

,PermitNumber,OFTCode,OnStreetName,FromStreetName,ToStreetName,BoroughName,PermitIssueDate,PermitLinearFeet,ApplicationTypeShortDesc,PermitSeriesShortDesc,On_Code,From_Code,From_On,Date,Year,Month,index_right,GEOID,COUNTYFP,geometry
0,M012019112A85,110210118630118656,3 AVENUE,EAST 82 STREET,EAST 83 STREET,MANHATTAN,04/22/2019 09:59:18 AM,4.0,New,STREET OPENING PERMIT,110210,118630,POINT (-73.95587 40.77624),2019-04-22,2019,4,4361,360610140001,061,"POLYGON ((-73.95701 40.77754, -73.95655 40.778..."
1,M012017159B99,118630124890127792,EAST 82 STREET,LEXINGTON AVENUE,PARK AVENUE,MANHATTAN,06/12/2017 02:13:21 PM,100.0,Emergency,STREET OPENING PERMIT,118630,124890,POINT (-73.95747 40.77691),2017-06-12,2017,6,4361,360610140001,061,"POLYGON ((-73.95701 40.77754, -73.95655 40.778..."
2,M012017160D00,118630124890127792,EAST 82 STREET,LEXINGTON AVENUE,PARK AVENUE,MANHATTAN,07/26/2017 11:09:51 AM,10.0,Emergency,STREET OPENING PERMIT,118630,124890,POINT (-73.95747 40.77691),2017-07-26,2017,7,4361,360610140001,061,"POLYGON ((-73.95701 40.77754, -73.95655 40.778..."
3,M012017188A09,118630110210124896,EAST 82 STREET,3 AVENUE,LEXINGTON AVENUE,MANHATTAN,07/12/2017 04:41:30 PM,4.0,Emergency,STREET OPENING PERMIT,118630,110210,POINT (-73.95587 40.77624),2017-07-12,2017,7,4361,360610140001,061,"POLYGON ((-73.95701 40.77754, -73.95655 40.778..."
4,M012017191A66,118630110210124896,EAST 82 STREET,3 AVENUE,LEXINGTON AVENUE,MANHATTAN,07/26/2017 10:34:17 AM,4.0,Emergency,STREET OPENING PERMIT,118630,110210,POINT (-73.95587 40.77624),2017-07-26,2017,7,4361,360610140001,061,"POLYGON ((-73.95701 40.77754, -73.95655 40.778..."


In [ ]:
permits_bg_polygon_gas["Year"].value_counts()

2019    88945
2022    42090
2018    13694
2020     7828
2021     5225
2017      894
2023       17
2015        7
2016        7
Name: Year, dtype: int64

In [ ]:
permits_bg_polygon_gas = gpd.GeoDataFrame(permits_bg_polygon_gas,geometry="geometry")
permits_bg_polygon_gas = permits_bg_polygon_gas.to_crs(bg.crs)

In [ ]:
permits_bg_polygon_gas = permits_bg_polygon_gas.drop('From_On', axis=1)

In [ ]:
permits_bg_polygon_gas.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
Int64Index: 158707 entries, 0 to 158706
Data columns (total 19 columns):
 #   Column                    Non-Null Count   Dtype         
---  ------                    --------------   -----         
 0   PermitNumber              158707 non-null  object        
 1   OFTCode                   158707 non-null  object        
 2   OnStreetName              158707 non-null  object        
 3   FromStreetName            158704 non-null  object        
 4   ToStreetName              139390 non-null  object        
 5   BoroughName               158707 non-null  object        
 6   PermitIssueDate           158707 non-null  object        
 7   PermitLinearFeet          158673 non-null  float64       
 8   ApplicationTypeShortDesc  158707 non-null  object        
 9   PermitSeriesShortDesc     158707 non-null  object        
 10  On_Code                   158707 non-null  object        
 11  From_Code                 158707 non-null  object        

In [ ]:
permits_bg_polygon_gas["Date"] = permits_bg_polygon_gas["Date"].astype("str")

In [ ]:
#permits_bg_polygon_gas.to_file("permits_bg_polygon_gas.shp")

In [ ]:
#bg.to_file("bg_nyc.shp")

In [ ]:
#permits_bg_polygon_gas.to_csv("permits_bg_polygon_gas.csv")

In [ ]:
permits_bg_polygon_gas.head()

,PermitNumber,OFTCode,OnStreetName,FromStreetName,ToStreetName,BoroughName,PermitIssueDate,PermitLinearFeet,ApplicationTypeShortDesc,PermitSeriesShortDesc,On_Code,From_Code,Date,Year,Month,index_right,GEOID,COUNTYFP,geometry
0,M012019112A85,110210118630118656,3 AVENUE,EAST 82 STREET,EAST 83 STREET,MANHATTAN,04/22/2019 09:59:18 AM,4.0,New,STREET OPENING PERMIT,110210,118630,2019-04-22,2019,4,4361,360610140001,061,"POLYGON ((-73.95701 40.77754, -73.95655 40.778..."
1,M012017159B99,118630124890127792,EAST 82 STREET,LEXINGTON AVENUE,PARK AVENUE,MANHATTAN,06/12/2017 02:13:21 PM,100.0,Emergency,STREET OPENING PERMIT,118630,124890,2017-06-12,2017,6,4361,360610140001,061,"POLYGON ((-73.95701 40.77754, -73.95655 40.778..."
2,M012017160D00,118630124890127792,EAST 82 STREET,LEXINGTON AVENUE,PARK AVENUE,MANHATTAN,07/26/2017 11:09:51 AM,10.0,Emergency,STREET OPENING PERMIT,118630,124890,2017-07-26,2017,7,4361,360610140001,061,"POLYGON ((-73.95701 40.77754, -73.95655 40.778..."
3,M012017188A09,118630110210124896,EAST 82 STREET,3 AVENUE,LEXINGTON AVENUE,MANHATTAN,07/12/2017 04:41:30 PM,4.0,Emergency,STREET OPENING PERMIT,118630,110210,2017-07-12,2017,7,4361,360610140001,061,"POLYGON ((-73.95701 40.77754, -73.95655 40.778..."
4,M012017191A66,118630110210124896,EAST 82 STREET,3 AVENUE,LEXINGTON AVENUE,MANHATTAN,07/26/2017 10:34:17 AM,4.0,Emergency,STREET OPENING PERMIT,118630,110210,2017-07-26,2017,7,4361,360610140001,061,"POLYGON ((-73.95701 40.77754, -73.95655 40.778..."


### Fill missing values

In [ ]:
permits_bg_polygon_gas = pd.read_csv("G:/Independent Study/CSV_Exports/permits_bg_polygon_gas.csv")
permits_bg_polygon_gas.head()

,Unnamed: 0,PermitNumber,OFTCode,OnStreetName,FromStreetName,ToStreetName,BoroughName,PermitIssueDate,PermitLinearFeet,ApplicationTypeShortDesc,PermitSeriesShortDesc,On_Code,From_Code,Date,Year,Month,index_right,GEOID,COUNTYFP,geometry
0,0,M012019112A85,110210118630118656,3 AVENUE,EAST 82 STREET,EAST 83 STREET,MANHATTAN,04/22/2019 09:59:18 AM,4.0,New,STREET OPENING PERMIT,110210,118630,2019-04-22,2019,4,4361,360610140001,61,"POLYGON ((-73.95701 40.777544999999996, -73.95..."
1,1,M012017159B99,118630124890127792,EAST 82 STREET,LEXINGTON AVENUE,PARK AVENUE,MANHATTAN,06/12/2017 02:13:21 PM,100.0,Emergency,STREET OPENING PERMIT,118630,124890,2017-06-12,2017,6,4361,360610140001,61,"POLYGON ((-73.95701 40.777544999999996, -73.95..."
2,2,M012017160D00,118630124890127792,EAST 82 STREET,LEXINGTON AVENUE,PARK AVENUE,MANHATTAN,07/26/2017 11:09:51 AM,10.0,Emergency,STREET OPENING PERMIT,118630,124890,2017-07-26,2017,7,4361,360610140001,61,"POLYGON ((-73.95701 40.777544999999996, -73.95..."
3,3,M012017188A09,118630110210124896,EAST 82 STREET,3 AVENUE,LEXINGTON AVENUE,MANHATTAN,07/12/2017 04:41:30 PM,4.0,Emergency,STREET OPENING PERMIT,118630,110210,2017-07-12,2017,7,4361,360610140001,61,"POLYGON ((-73.95701 40.777544999999996, -73.95..."
4,4,M012017191A66,118630110210124896,EAST 82 STREET,3 AVENUE,LEXINGTON AVENUE,MANHATTAN,07/26/2017 10:34:17 AM,4.0,Emergency,STREET OPENING PERMIT,118630,110210,2017-07-26,2017,7,4361,360610140001,61,"POLYGON ((-73.95701 40.777544999999996, -73.95..."


In [ ]:
permits_bg_polygon_gas.shape

(158707, 20)

In [ ]:
permits_bg_polygon_gas = permits_bg_polygon_gas.drop('Unnamed: 0', axis=1)
permits_bg_polygon_gas.head()

,PermitNumber,OFTCode,OnStreetName,FromStreetName,ToStreetName,BoroughName,PermitIssueDate,PermitLinearFeet,ApplicationTypeShortDesc,PermitSeriesShortDesc,On_Code,From_Code,Date,Year,Month,index_right,GEOID,COUNTYFP,geometry
0,M012019112A85,110210118630118656,3 AVENUE,EAST 82 STREET,EAST 83 STREET,MANHATTAN,04/22/2019 09:59:18 AM,4.0,New,STREET OPENING PERMIT,110210,118630,2019-04-22,2019,4,4361,360610140001,61,"POLYGON ((-73.95701 40.777544999999996, -73.95..."
1,M012017159B99,118630124890127792,EAST 82 STREET,LEXINGTON AVENUE,PARK AVENUE,MANHATTAN,06/12/2017 02:13:21 PM,100.0,Emergency,STREET OPENING PERMIT,118630,124890,2017-06-12,2017,6,4361,360610140001,61,"POLYGON ((-73.95701 40.777544999999996, -73.95..."
2,M012017160D00,118630124890127792,EAST 82 STREET,LEXINGTON AVENUE,PARK AVENUE,MANHATTAN,07/26/2017 11:09:51 AM,10.0,Emergency,STREET OPENING PERMIT,118630,124890,2017-07-26,2017,7,4361,360610140001,61,"POLYGON ((-73.95701 40.777544999999996, -73.95..."
3,M012017188A09,118630110210124896,EAST 82 STREET,3 AVENUE,LEXINGTON AVENUE,MANHATTAN,07/12/2017 04:41:30 PM,4.0,Emergency,STREET OPENING PERMIT,118630,110210,2017-07-12,2017,7,4361,360610140001,61,"POLYGON ((-73.95701 40.777544999999996, -73.95..."
4,M012017191A66,118630110210124896,EAST 82 STREET,3 AVENUE,LEXINGTON AVENUE,MANHATTAN,07/26/2017 10:34:17 AM,4.0,Emergency,STREET OPENING PERMIT,118630,110210,2017-07-26,2017,7,4361,360610140001,61,"POLYGON ((-73.95701 40.777544999999996, -73.95..."


In [ ]:
permits_bg_polygon_gas["Year"].value_counts()

2019    88945
2022    42090
2018    13694
2020     7828
2021     5225
2017      894
2023       17
2015        7
2016        7
Name: Year, dtype: int64

In [ ]:
permits_bg_polygon_gas.head()

,PermitNumber,OFTCode,OnStreetName,FromStreetName,ToStreetName,BoroughName,PermitIssueDate,PermitLinearFeet,ApplicationTypeShortDesc,PermitSeriesShortDesc,On_Code,From_Code,Date,Year,Month,index_right,GEOID,COUNTYFP,geometry
0,M012019112A85,110210118630118656,3 AVENUE,EAST 82 STREET,EAST 83 STREET,MANHATTAN,04/22/2019 09:59:18 AM,4.0,New,STREET OPENING PERMIT,110210,118630,2019-04-22,2019,4,4361,360610140001,61,"POLYGON ((-73.95701 40.777544999999996, -73.95..."
1,M012017159B99,118630124890127792,EAST 82 STREET,LEXINGTON AVENUE,PARK AVENUE,MANHATTAN,06/12/2017 02:13:21 PM,100.0,Emergency,STREET OPENING PERMIT,118630,124890,2017-06-12,2017,6,4361,360610140001,61,"POLYGON ((-73.95701 40.777544999999996, -73.95..."
2,M012017160D00,118630124890127792,EAST 82 STREET,LEXINGTON AVENUE,PARK AVENUE,MANHATTAN,07/26/2017 11:09:51 AM,10.0,Emergency,STREET OPENING PERMIT,118630,124890,2017-07-26,2017,7,4361,360610140001,61,"POLYGON ((-73.95701 40.777544999999996, -73.95..."
3,M012017188A09,118630110210124896,EAST 82 STREET,3 AVENUE,LEXINGTON AVENUE,MANHATTAN,07/12/2017 04:41:30 PM,4.0,Emergency,STREET OPENING PERMIT,118630,110210,2017-07-12,2017,7,4361,360610140001,61,"POLYGON ((-73.95701 40.777544999999996, -73.95..."
4,M012017191A66,118630110210124896,EAST 82 STREET,3 AVENUE,LEXINGTON AVENUE,MANHATTAN,07/26/2017 10:34:17 AM,4.0,Emergency,STREET OPENING PERMIT,118630,110210,2017-07-26,2017,7,4361,360610140001,61,"POLYGON ((-73.95701 40.777544999999996, -73.95..."


#### Years 2018-2022

In [ ]:
permits_bg_polygon_gas.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 158707 entries, 0 to 158706
Data columns (total 19 columns):
 #   Column                    Non-Null Count   Dtype  
---  ------                    --------------   -----  
 0   PermitNumber              158707 non-null  object 
 1   OFTCode                   158707 non-null  int64  
 2   OnStreetName              158707 non-null  object 
 3   FromStreetName            158704 non-null  object 
 4   ToStreetName              139390 non-null  object 
 5   BoroughName               158707 non-null  object 
 6   PermitIssueDate           158707 non-null  object 
 7   PermitLinearFeet          158673 non-null  float64
 8   ApplicationTypeShortDesc  158707 non-null  object 
 9   PermitSeriesShortDesc     158707 non-null  object 
 10  On_Code                   158707 non-null  int64  
 11  From_Code                 158707 non-null  int64  
 12  Date                      158707 non-null  object 
 13  Year                      158707 non-null  i

In [ ]:
permits_years = permits_bg_polygon_gas[(permits_bg_polygon_gas["Year"]>=2018) & (permits_bg_polygon_gas["Year"]<=2022)]
permits_years["Year"].value_counts()

2019    88945
2022    42090
2018    13694
2020     7828
2021     5225
Name: Year, dtype: int64

In [ ]:
permits_years["Year"].value_counts()

2019    88945
2022    42090
2018    13694
2020     7828
2021     5225
Name: Year, dtype: int64

In [ ]:
permits_years["Count"] = 1

In [ ]:
permits_years.head()

,PermitNumber,OFTCode,OnStreetName,FromStreetName,ToStreetName,BoroughName,PermitIssueDate,PermitLinearFeet,ApplicationTypeShortDesc,PermitSeriesShortDesc,On_Code,From_Code,Date,Year,Month,index_right,GEOID,COUNTYFP,geometry,Count
0,M012019112A85,110210118630118656,3 AVENUE,EAST 82 STREET,EAST 83 STREET,MANHATTAN,04/22/2019 09:59:18 AM,4.0,New,STREET OPENING PERMIT,110210,118630,2019-04-22,2019,4,4361,360610140001,61,"POLYGON ((-73.95701 40.777544999999996, -73.95...",1
13,M012018003C14,118630124890127792,EAST 82 STREET,LEXINGTON AVENUE,PARK AVENUE,MANHATTAN,01/03/2018 02:16:43 PM,45.0,New,STREET OPENING PERMIT,118630,124890,2018-01-03,2018,1,4361,360610140001,61,"POLYGON ((-73.95701 40.777544999999996, -73.95...",1
14,M012018004A00,118650124890000000,EAST 83 STREET,LEXINGTON AVENUE,NaN,MANHATTAN,01/09/2018 12:30:54 PM,20.0,Emergency,STREET OPENING PERMIT,118650,124890,2018-01-09,2018,1,4361,360610140001,61,"POLYGON ((-73.95701 40.777544999999996, -73.95...",1
15,M012018008A68,118650124890000000,EAST 83 STREET,LEXINGTON AVENUE,NaN,MANHATTAN,01/09/2018 10:56:11 AM,4.0,Emergency,STREET OPENING PERMIT,118650,124890,2018-01-09,2018,1,4361,360610140001,61,"POLYGON ((-73.95701 40.777544999999996, -73.95...",1
16,M012018022B72,124890118630118656,LEXINGTON AVENUE,EAST 82 STREET,EAST 83 STREET,MANHATTAN,01/24/2018 10:06:38 AM,4.0,Emergency,STREET OPENING PERMIT,124890,118630,2018-01-24,2018,1,4361,360610140001,61,"POLYGON ((-73.95701 40.777544999999996, -73.95...",1


In [ ]:
bg_nyc.shape

(6493, 13)

In [ ]:
permits_years.GEOID.nunique()

5431

In [ ]:
bg_nyc.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
Int64Index: 6493 entries, 0 to 15341
Data columns (total 13 columns):
 #   Column    Non-Null Count  Dtype   
---  ------    --------------  -----   
 0   STATEFP   6493 non-null   object  
 1   COUNTYFP  6493 non-null   object  
 2   TRACTCE   6493 non-null   object  
 3   BLKGRPCE  6493 non-null   object  
 4   GEOID     6493 non-null   object  
 5   NAMELSAD  6493 non-null   object  
 6   MTFCC     6493 non-null   object  
 7   FUNCSTAT  6493 non-null   object  
 8   ALAND     6493 non-null   int64   
 9   AWATER    6493 non-null   int64   
 10  INTPTLAT  6493 non-null   object  
 11  INTPTLON  6493 non-null   object  
 12  geometry  6493 non-null   geometry
dtypes: geometry(1), int64(2), object(10)
memory usage: 710.2+ KB


In [ ]:
permits_years.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 157782 entries, 0 to 158706
Data columns (total 20 columns):
 #   Column                    Non-Null Count   Dtype  
---  ------                    --------------   -----  
 0   PermitNumber              157782 non-null  object 
 1   OFTCode                   157782 non-null  int64  
 2   OnStreetName              157782 non-null  object 
 3   FromStreetName            157780 non-null  object 
 4   ToStreetName              138615 non-null  object 
 5   BoroughName               157782 non-null  object 
 6   PermitIssueDate           157782 non-null  object 
 7   PermitLinearFeet          157748 non-null  float64
 8   ApplicationTypeShortDesc  157782 non-null  object 
 9   PermitSeriesShortDesc     157782 non-null  object 
 10  On_Code                   157782 non-null  int64  
 11  From_Code                 157782 non-null  int64  
 12  Date                      157782 non-null  object 
 13  Year                      157782 non-null  i

In [ ]:
permits_years["GEOID"] = permits_years["GEOID"].astype(str)

In [ ]:
bg_nyc_new = bg_nyc[bg_nyc["GEOID"].isin(permits_years.GEOID)]
bg_nyc_new.shape

(5431, 13)

In [ ]:
bg_nyc_new.to_file("bg_nyc_new.shp")

In [ ]:
#permits_years.to_csv("G:/Independent Study/CSV_Exports/permits_2018-2022.csv")

In [ ]:
permits_years_19_22 = permits_years[(permits_years["Year"]==2019) | (permits_years["Year"]==2022)]
permits_years_19_22["Year"].value_counts()

2019    88945
2022    42090
Name: Year, dtype: int64

In [ ]:
permits_years_19_22["Count"] = 1

In [ ]:
#permits_years_19_22.to_csv("permits_years_19_22.csv")

In [ ]:
permits_years_2019 = permits_years[permits_years["Year"]==2019]
permits_years_2019.head()

,PermitNumber,OFTCode,OnStreetName,FromStreetName,ToStreetName,BoroughName,PermitIssueDate,PermitLinearFeet,ApplicationTypeShortDesc,PermitSeriesShortDesc,On_Code,From_Code,Date,Year,Month,index_right,GEOID,COUNTYFP,geometry,Count
0,M012019112A85,110210118630118656,3 AVENUE,EAST 82 STREET,EAST 83 STREET,MANHATTAN,04/22/2019 09:59:18 AM,4.0,New,STREET OPENING PERMIT,110210,118630,2019-04-22,2019,4,4361,360610140001,61,"POLYGON ((-73.95701 40.777544999999996, -73.95...",1
25,M012019058B81,118630110210124896,EAST 82 STREET,3 AVENUE,LEXINGTON AVENUE,MANHATTAN,02/27/2019 10:50:02 AM,4.0,New,STREET OPENING PERMIT,118630,110210,2019-02-27,2019,2,4361,360610140001,61,"POLYGON ((-73.95701 40.777544999999996, -73.95...",1
26,M012019067B35,110210118670118688,3 AVENUE,EAST 84 STREET,EAST 85 STREET,MANHATTAN,03/08/2019 01:20:51 PM,240.0,New,STREET OPENING PERMIT,110210,118670,2019-03-08,2019,3,4361,360610140001,61,"POLYGON ((-73.95701 40.777544999999996, -73.95...",1
27,M012019108A31,110210118630118656,3 AVENUE,EAST 82 STREET,EAST 83 STREET,MANHATTAN,04/18/2019 09:30:24 AM,4.0,New,STREET OPENING PERMIT,110210,118630,2019-04-18,2019,4,4361,360610140001,61,"POLYGON ((-73.95701 40.777544999999996, -73.95...",1
28,M012019127A48,110210118630118656,3 AVENUE,EAST 82 STREET,EAST 83 STREET,MANHATTAN,05/07/2019 07:39:53 AM,4.0,New,STREET OPENING PERMIT,110210,118630,2019-05-07,2019,5,4361,360610140001,61,"POLYGON ((-73.95701 40.777544999999996, -73.95...",1


In [ ]:
#permits_years_2019.to_csv("permits_years_2019.csv")

### Income

In [ ]:
bronx_income = pd.read_csv("C:/Users/as16477/Desktop/Independent Study/Bronx/acs2021_5yr_B19013_15000US360050019040/acs2021_5yr_B19013_15000US360050019040.csv")
bronx_income["Borough"] = "Bronx"
bronx_income['geoid'] = bronx_income['geoid'].apply(lambda x: x.replace('15000US', ''))
bronx_income = bronx_income[["geoid","B19013001","Borough"]].rename(columns={"geoid":"GEOID","B19013001":"Median Income"})
bronx_income.head()

,GEOID,Median Income,Borough
0,360050001000,NaN,Bronx
1,360050001001,NaN,Bronx
2,360050002000,NaN,Bronx
3,360050002001,NaN,Bronx
4,360050002002,72232.0,Bronx


In [ ]:
man_income = pd.read_csv("C:/Users/as16477/Desktop/Independent Study/acs2021_5yr_B19013_15000US360610182003/acs2021_5yr_B19013_15000US360610182003/acs2021_5yr_B19013_15000US360610182003.csv")
man_income["Borough"] = "Manhattan"
man_income['geoid'] = man_income['geoid'].apply(lambda x: x.replace('15000US', ''))
man_income = man_income[["geoid","B19013001","Borough"]].rename(columns={"geoid":"GEOID","B19013001":"Median Income"})
man_income.head()

,GEOID,Median Income,Borough
0,360610001001,NaN,Manhattan
1,360610002010,NaN,Manhattan
2,360610002011,59773.0,Manhattan
3,360610002012,13362.0,Manhattan
4,360610002020,NaN,Manhattan


In [ ]:
LI_income = pd.read_csv("C:/Users/as16477/Desktop/Independent Study/LI_new/acs2021_5yr_B19013_15000US360594092001/acs2021_5yr_B19013_15000US360594092001.csv")
LI_income["Borough"] = "Long Island"
LI_income['geoid'] = LI_income['geoid'].apply(lambda x: x.replace('15000US', ''))
LI_income = LI_income[["geoid","B19013001","Borough"]].rename(columns={"geoid":"GEOID","B19013001":"Median Income"})
LI_income.head()

,GEOID,Median Income,Borough
0,360593001001,159605.0,Long Island
1,360593001002,150000.0,Long Island
2,360593001003,131797.0,Long Island
3,360593001004,176607.0,Long Island
4,360593003001,NaN,Long Island


In [ ]:
Br_income = pd.read_csv("C:/Users/as16477/Desktop/Independent Study/Br_New/acs2021_5yr_B19013_15000US360470248002/acs2021_5yr_B19013_15000US360470248002.csv")
Br_income["Borough"] = "Brooklyn"
Br_income['geoid'] = Br_income['geoid'].apply(lambda x: x.replace('15000US', ''))
Br_income = Br_income[["geoid","B19013001","Borough"]].rename(columns={"geoid":"GEOID","B19013001":"Median Income"})
Br_income.head()

,GEOID,Median Income,Borough
0,360470001001,250001.0,Brooklyn
1,360470001002,111184.0,Brooklyn
2,360470001003,162813.0,Brooklyn
3,360470001004,102581.0,Brooklyn
4,360470002001,58882.0,Brooklyn


In [ ]:
Qu_income = pd.read_csv("C:/Users/as16477/Desktop/Independent Study/Queens_Income/acs2021_5yr_B19013_15000US360810110002/acs2021_5yr_B19013_15000US360810110002.csv")
Qu_income["Borough"] = "Queens"
Qu_income['geoid'] = Qu_income['geoid'].apply(lambda x: x.replace('15000US', ''))
Qu_income = Qu_income[["geoid","B19013001","Borough"]].rename(columns={"geoid":"GEOID","B19013001":"Median Income"})
Qu_income.head()

,GEOID,Median Income,Borough
0,360810001011,119698.0,Queens
1,360810001012,96250.0,Queens
2,360810001021,176295.0,Queens
3,360810001022,199489.0,Queens
4,360810001031,NaN,Queens


In [ ]:
income_ny = pd.concat([bronx_income, man_income, LI_income, Br_income, Qu_income], ignore_index=True)
income_ny.head()

,GEOID,Median Income,Borough
0,360050001000,NaN,Bronx
1,360050001001,NaN,Bronx
2,360050002000,NaN,Bronx
3,360050002001,NaN,Bronx
4,360050002002,72232.0,Bronx


In [ ]:
income_df = income_ny.dropna(subset=['Median Income'])

In [ ]:
income_df.head()

,GEOID,Median Income,Borough
4,360050002002,72232.0,Bronx
5,360050002003,72286.0,Bronx
8,360050004002,64616.0,Bronx
9,360050004003,99361.0,Bronx
10,360050004004,99262.0,Bronx


In [ ]:
quantiles = [0, 0.33, 0.67, 1.0]

income_df['Income_Bin'] = pd.qcut(income_df['Median Income'], q=quantiles, labels=False)

# Map bin labels to meaningful names
income_df['Income_Bin'] = income_df['Income_Bin'].map({0: 'Low', 1: 'Medium', 2: 'High'})
income_df.head()

,GEOID,Median Income,Borough,Income_Bin
4,360050002002,72232.0,Bronx,Medium
5,360050002003,72286.0,Bronx,Medium
8,360050004002,64616.0,Bronx,Medium
9,360050004003,99361.0,Bronx,Medium
10,360050004004,99262.0,Bronx,Medium


In [ ]:
income_df["Income_Bin"].value_counts()

Medium    2168
Low       2104
High      2104
Name: Income_Bin, dtype: int64

### Emerging Hotspot Analysis

#### Count

In [ ]:
df = pd.read_csv("count.csv")

In [ ]:
df.head()

,OBJECTID *,Location ID,Location Label,Category,Pattern Type COUNT_NONE_ZEROS,Percent Significant Hot Spot,Percent Significant Cold Spot,Trend z-score COUNT_NONE_ZEROS,Trend p-value COUNT_NONE_ZEROS,Trend bin COUNT_NONE_ZEROS,Sum COUNT_NONE_ZEROS,Min COUNT_NONE_ZEROS,Max COUNT_NONE_ZEROS,Mean COUNT_NONE_ZEROS,Standard Deviation COUNT_NONE_ZEROS,Median COUNT_NONE_ZEROS,Shape_Length,Shape_Area
0,1,0,360050418001,6,Sporadic Hot Spot,25,0,1.245384,0.212991,0,1,0,1,0.05,0.217945,0,7302.756491,1.515446e+06
1,2,1,360050127012,7,Oscillating Hot Spot,30,35,1.397157,0.162366,0,2,0,1,0.10,0.300000,0,3351.112700,7.263214e+05
2,3,2,360050211003,7,Oscillating Hot Spot,25,40,1.247635,0.212165,0,1,0,1,0.05,0.217945,0,2693.020866,4.547823e+05
3,4,3,360050255003,0,No Pattern Detected,30,45,0.655708,0.512012,0,1,0,1,0.05,0.217945,0,4592.182219,6.654805e+05
4,5,4,360811163005,0,No Pattern Detected,0,0,0.000000,1.000000,0,3,0,1,0.15,0.357071,0,3428.092828,6.797547e+05


In [ ]:
df["Location Label"] = df["Location Label"].astype(str)

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 436 entries, 0 to 435
Data columns (total 18 columns):
 #   Column                               Non-Null Count  Dtype  
---  ------                               --------------  -----  
 0   OBJECTID *                           436 non-null    int64  
 1   Location ID                          436 non-null    int64  
 2   Location Label                       436 non-null    object 
 3   Category                             436 non-null    int64  
 4   Pattern Type COUNT_NONE_ZEROS        436 non-null    object 
 5   Percent Significant Hot Spot         436 non-null    int64  
 6   Percent Significant Cold Spot        436 non-null    int64  
 7   Trend z-score COUNT_NONE_ZEROS       436 non-null    float64
 8   Trend p-value COUNT_NONE_ZEROS       436 non-null    float64
 9   Trend bin COUNT_NONE_ZEROS           436 non-null    int64  
 10  Sum COUNT_NONE_ZEROS                 436 non-null    int64  
 11  Min COUNT_NONE_ZEROS            

In [ ]:
df_count = pd.merge(df[["Location Label","Pattern Type COUNT_NONE_ZEROS","Percent Significant Hot Spot","Percent Significant Cold Spot"]],
                   income_df,left_on="Location Label",right_on="GEOID")
df_count.head()

,Location Label,Pattern Type COUNT_NONE_ZEROS,Percent Significant Hot Spot,Percent Significant Cold Spot,GEOID,Median Income,Borough,Income_Bin
0,360050127012,Oscillating Hot Spot,30,35,360050127012,53846.0,Bronx,Low
1,360050211003,Oscillating Hot Spot,25,40,360050211003,14892.0,Bronx,Low
2,360050086002,Sporadic Hot Spot,30,0,360050086002,26250.0,Bronx,Low
3,360810998012,No Pattern Detected,0,0,360810998012,64604.0,Queens,Medium
4,360470770001,No Pattern Detected,15,25,360470770001,118902.0,Brooklyn,High


In [ ]:
df_count.shape

(321, 8)

In [ ]:
df_count["Location Label"].nunique()

321

In [ ]:
df_count["Borough"].value_counts()

Brooklyn     102
Manhattan     91
Bronx         73
Queens        55
Name: Borough, dtype: int64

In [ ]:
pd.pivot_table(df_count,index=['Borough'], columns='Pattern Type COUNT_NONE_ZEROS', values='Location Label', aggfunc='count')

Pattern Type COUNT_NONE_ZEROS,Consecutive Hot Spot,No Pattern Detected,Oscillating Hot Spot,Sporadic Hot Spot
Borough,,,,
Bronx,2.0,32.0,28.0,11.0
Brooklyn,1.0,67.0,21.0,13.0
Manhattan,NaN,44.0,47.0,NaN
Queens,2.0,36.0,9.0,8.0


In [ ]:
df_count_hot = df_count[df_count["Pattern Type COUNT_NONE_ZEROS"]!="No Pattern Detected"]
df_count_hot.head()

,Location Label,Pattern Type COUNT_NONE_ZEROS,Percent Significant Hot Spot,Percent Significant Cold Spot,GEOID,Median Income,Borough,Income_Bin
0,360050127012,Oscillating Hot Spot,30,35,360050127012,53846.0,Bronx,Low
1,360050211003,Oscillating Hot Spot,25,40,360050211003,14892.0,Bronx,Low
2,360050086002,Sporadic Hot Spot,30,0,360050086002,26250.0,Bronx,Low
6,360050378002,Oscillating Hot Spot,30,40,360050378002,65385.0,Bronx,Medium
8,360050393001,Oscillating Hot Spot,30,45,360050393001,39879.0,Bronx,Low


In [ ]:
df_count_hot["Income_Bin"].value_counts()

Low       62
Medium    43
High      37
Name: Income_Bin, dtype: int64

#### Length

In [ ]:
length = pd.read_csv("Length.csv")
length.head()

,OBJECTID *,Shape *,Location ID,Location Label,Category,Pattern Type PERMITLINEARFEET_NONE_ZEROS,Percent Significant Hot Spot,Percent Significant Cold Spot,Trend z-score PERMITLINEARFEET_NONE_ZEROS,Trend p-value PERMITLINEARFEET_NONE_ZEROS,Trend bin PERMITLINEARFEET_NONE_ZEROS,Sum PERMITLINEARFEET_NONE_ZEROS,Min PERMITLINEARFEET_NONE_ZEROS,Max PERMITLINEARFEET_NONE_ZEROS,Mean PERMITLINEARFEET_NONE_ZEROS,Standard Deviation PERMITLINEARFEET_NONE_ZEROS,Median PERMITLINEARFEET_NONE_ZEROS,Shape_Length,Shape_Area
0,1,Polygon,0,360050418001,0,No Pattern Detected,0,0,2.018399,0.043550,2,60,0,60,3.00,13.076697,0,7302.756491,1.515446e+06
1,2,Polygon,1,360050127012,0,No Pattern Detected,0,0,1.762104,0.078052,1,44,0,22,2.20,6.600000,0,3351.112700,7.263214e+05
2,3,Polygon,2,360050211003,0,No Pattern Detected,0,0,1.244509,0.213312,0,20,0,20,1.00,4.358899,0,2693.020866,4.547823e+05
3,4,Polygon,3,360050255003,0,No Pattern Detected,0,0,0.589504,0.555523,0,55,0,55,2.75,11.986972,0,4592.182219,6.654805e+05
4,5,Polygon,4,360811163005,0,No Pattern Detected,0,0,0.931438,0.351627,0,151,0,56,7.55,18.040164,0,3428.092828,6.797547e+05


In [ ]:
length.shape

(436, 19)

In [ ]:
length["Location Label"] = length["Location Label"].astype(str)

In [ ]:
df_length = pd.merge(income_df,length[["Location Label","Pattern Type PERMITLINEARFEET_NONE_ZEROS","Percent Significant Hot Spot",
                                          "Percent Significant Cold Spot"]],left_on="GEOID",right_on="Location Label")

df_length.head()

,GEOID,Median Income,Borough,Income_Bin,Location Label,Pattern Type PERMITLINEARFEET_NONE_ZEROS,Percent Significant Hot Spot,Percent Significant Cold Spot
0,360050025002,18152.0,Bronx,Low,360050025002,No Pattern Detected,0,0
1,360050033003,15313.0,Bronx,Low,360050033003,No Pattern Detected,0,0
2,360050035003,32233.0,Bronx,Low,360050035003,No Pattern Detected,0,0
3,360050038001,83295.0,Bronx,Medium,360050038001,No Pattern Detected,0,0
4,360050042002,11629.0,Bronx,Low,360050042002,No Pattern Detected,0,0


In [ ]:
df_length.shape

(321, 8)

In [ ]:
df_length.nunique()

GEOID                                       321
Median Income                               316
Borough                                       4
Income_Bin                                    3
Location Label                              321
Pattern Type PERMITLINEARFEET_NONE_ZEROS      4
Percent Significant Hot Spot                  7
Percent Significant Cold Spot                 1
dtype: int64

In [ ]:
pd.pivot_table(df_length,index=['Borough'], columns='Pattern Type PERMITLINEARFEET_NONE_ZEROS', values='Location Label', aggfunc='count')

Pattern Type PERMITLINEARFEET_NONE_ZEROS,Consecutive Hot Spot,New Hot Spot,No Pattern Detected,Sporadic Hot Spot
Borough,,,,
Bronx,NaN,NaN,72.0,1.0
Brooklyn,17.0,1.0,76.0,8.0
Manhattan,NaN,NaN,91.0,NaN
Queens,8.0,NaN,36.0,11.0


In [ ]:
income_ny.head()

,GEOID,Median Income,Borough
0,360050001000,NaN,Bronx
1,360050001001,NaN,Bronx
2,360050002000,NaN,Bronx
3,360050002001,NaN,Bronx
4,360050002002,72232.0,Bronx


In [ ]:
income_ny["Median Income"].median()

83075.5

In [ ]:
df_length_hot = df_length[df_length["Pattern Type PERMITLINEARFEET_NONE_ZEROS"]!="No Pattern Detected"]
df_length_hot.head()

,GEOID,Median Income,Borough,Income_Bin,Location Label,Pattern Type PERMITLINEARFEET_NONE_ZEROS,Percent Significant Hot Spot,Percent Significant Cold Spot
13,360050096001,55066.0,Bronx,Low,360050096001,Sporadic Hot Spot,10,0
170,360470100002,85417.0,Brooklyn,Medium,360470100002,Consecutive Hot Spot,10,0
174,360470136002,126307.0,Brooklyn,High,360470136002,Consecutive Hot Spot,10,0
188,360470271002,99375.0,Brooklyn,Medium,360470271002,Sporadic Hot Spot,30,0
195,360470292002,42829.0,Brooklyn,Low,360470292002,Consecutive Hot Spot,10,0


In [ ]:
df_length_hot = df_length_hot.rename(columns={"GEOID":"Block Group"})
df_length_hot.head(20)

,Block Group,Median Income,Borough,Income_Bin,Location Label,Pattern Type PERMITLINEARFEET_NONE_ZEROS,Percent Significant Hot Spot,Percent Significant Cold Spot
13,360050096001,55066.0,Bronx,Low,360050096001,Sporadic Hot Spot,10,0
170,360470100002,85417.0,Brooklyn,Medium,360470100002,Consecutive Hot Spot,10,0
174,360470136002,126307.0,Brooklyn,High,360470136002,Consecutive Hot Spot,10,0
188,360470271002,99375.0,Brooklyn,Medium,360470271002,Sporadic Hot Spot,30,0
195,360470292002,42829.0,Brooklyn,Low,360470292002,Consecutive Hot Spot,10,0
196,360470297001,108146.0,Brooklyn,High,360470297001,Sporadic Hot Spot,20,0
197,360470298002,39306.0,Brooklyn,Low,360470298002,Consecutive Hot Spot,10,0
198,360470309002,110972.0,Brooklyn,High,360470309002,Sporadic Hot Spot,20,0
206,360470350002,41250.0,Brooklyn,Low,360470350002,Consecutive Hot Spot,10,0
207,360470354002,51964.0,Brooklyn,Low,360470354002,Consecutive Hot Spot,10,0


In [ ]:
df_length_hot["Income_Bin"].value_counts()

Low       25
Medium    17
High       4
Name: Income_Bin, dtype: int64

### Brooklyn and Queens has the highest gas infrastructure activity. Both served by National Grid!

#### Income

In [ ]:
income = pd.read_csv("income.csv")
income.head()

,geoid,name,B19013001,"B19013001, Error"
0,15000US360810001011,"BG 1, Tract 1.01, Queens, NY",119698.0,12324.0
1,15000US360810001012,"BG 2, Tract 1.01, Queens, NY",96250.0,67144.0
2,15000US360810001021,"BG 1, Tract 1.02, Queens, NY",176295.0,54663.0
3,15000US360810001022,"BG 2, Tract 1.02, Queens, NY",199489.0,24039.0
4,15000US360810001031,"BG 1, Tract 1.03, Queens, NY",NaN,NaN


In [ ]:
income['geoid'] = income['geoid'].apply(lambda x: x.replace('15000US', ''))

In [ ]:
income = income[["geoid","B19013001"]].rename(columns={"geoid":"GEOID","B19013001":"Median Income"})
income.head()

,GEOID,Median Income
0,360810001011,119698.0
1,360810001012,96250.0
2,360810001021,176295.0
3,360810001022,199489.0
4,360810001031,NaN


In [ ]:
df = pd.merge(bg_hotspot_pat,income,on="GEOID")
df.head()

,PermitNumber,OFTCode,OnStreetName,FromStreetName,ToStreetName,BoroughName,PermitIssueDate,PermitLinearFeet,ApplicationTypeShortDesc,PermitSeriesShortDesc,On_Code,From_Code,Date,Year,Month,index_right,GEOID,COUNTYFP,geometry,Count,Location Label,Pattern Type COUNT_NONE_ZEROS,Percent Significant Hot Spot,Percent Significant Cold Spot,Median Income
0,Q012018023B19,408790420740000000,37 AVENUE,114 STREET,NaN,QUEENS,01/23/2018 09:51:00 AM,20.0,Reissue,STREET OPENING PERMIT,408790,420740,2018-01-23,2018,1,8814,360810381003,81,"POLYGON ((-73.857928 40.756685, -73.857055 40....",1,360810381003,Oscillating Hot Spot,15,30,56625.0
1,Q012019022A48,420740412010457792,114 STREET,GRAND CENTRAL PY EB EN 34 AV,NORTHERN BOULEVARD,QUEENS,01/22/2019 09:17:49 AM,250.0,New,STREET OPENING PERMIT,420740,412010,2019-01-22,2019,1,8814,360810381003,81,"POLYGON ((-73.857928 40.756685, -73.857055 40....",1,360810381003,Oscillating Hot Spot,15,30,56625.0
2,Q012018215C04,415690435690000000,75 AVENUE,BELL BOULEVARD,NaN,QUEENS,08/03/2018 02:54:23 PM,5.0,New,STREET OPENING PERMIT,415690,435690,2018-08-03,2018,8,1645,360811291032,81,"POLYGON ((-73.762529 40.736004, -73.762485 40....",1,360811291032,Sporadic Hot Spot,20,0,72639.0
3,Q012018360B56,435690415690416064,BELL BOULEVARD,75 AVENUE,77 AVENUE,QUEENS,12/27/2018 01:27:16 PM,6.0,Emergency,STREET OPENING PERMIT,435690,415690,2018-12-27,2018,12,1645,360811291032,81,"POLYGON ((-73.762529 40.736004, -73.762485 40....",1,360811291032,Sporadic Hot Spot,20,0,72639.0
4,Q012020010B64,435690415690416064,BELL BOULEVARD,75 AVENUE,77 AVENUE,QUEENS,01/10/2020 11:14:16 AM,150.0,New,STREET OPENING PERMIT,435690,415690,2020-01-10,2020,1,1645,360811291032,81,"POLYGON ((-73.762529 40.736004, -73.762485 40....",1,360811291032,Sporadic Hot Spot,20,0,72639.0


In [ ]:
df = df[["GEOID","Pattern Type COUNT_NONE_ZEROS","Percent Significant Hot Spot","Percent Significant Cold Spot","Median Income","BoroughName"]]
df = df.drop_duplicates()
df.head()

,GEOID,Pattern Type COUNT_NONE_ZEROS,Percent Significant Hot Spot,Percent Significant Cold Spot,Median Income,BoroughName
0,360810381003,Oscillating Hot Spot,15,30,56625.0,QUEENS
2,360811291032,Sporadic Hot Spot,20,0,72639.0,QUEENS
7,360810413002,Oscillating Hot Spot,25,30,59306.0,QUEENS
10,360810339003,Sporadic Hot Spot,30,0,78906.0,QUEENS
13,360810713062,Sporadic Hot Spot,15,0,105771.0,QUEENS


In [ ]:
df["GEOID"].nunique()

21

In [ ]:
df

,GEOID,Pattern Type COUNT_NONE_ZEROS,Percent Significant Hot Spot,Percent Significant Cold Spot,Median Income,BoroughName
0,360810381003,Oscillating Hot Spot,15,30,56625.0,QUEENS
2,360811291032,Sporadic Hot Spot,20,0,72639.0,QUEENS
7,360810413002,Oscillating Hot Spot,25,30,59306.0,QUEENS
10,360810339003,Sporadic Hot Spot,30,0,78906.0,QUEENS
13,360810713062,Sporadic Hot Spot,15,0,105771.0,QUEENS
16,360810481003,Oscillating Hot Spot,25,30,NaN,QUEENS
17,360810437022,Oscillating Hot Spot,15,30,51331.0,QUEENS
19,360810467002,Oscillating Hot Spot,25,30,67083.0,QUEENS
24,360810269022,Oscillating Hot Spot,25,30,63433.0,QUEENS
27,360810339001,Sporadic Hot Spot,30,0,35064.0,QUEENS


### All hotspots in Queens!

#### Aggregating by Block Group

In [ ]:
52*2

104

In [ ]:
permits_years.head()

,PermitNumber,OFTCode,OnStreetName,FromStreetName,ToStreetName,BoroughName,PermitIssueDate,PermitLinearFeet,ApplicationTypeShortDesc,PermitSeriesShortDesc,On_Code,From_Code,Date,Year,Month,index_right,GEOID,COUNTYFP,geometry,Count
0,M012019112A85,110210118630118656,3 AVENUE,EAST 82 STREET,EAST 83 STREET,MANHATTAN,04/22/2019 09:59:18 AM,4.0,New,STREET OPENING PERMIT,110210,118630,2019-04-22,2019,4,4361,360610140001,61,"POLYGON ((-73.95701 40.777544999999996, -73.95...",1
13,M012018003C14,118630124890127792,EAST 82 STREET,LEXINGTON AVENUE,PARK AVENUE,MANHATTAN,01/03/2018 02:16:43 PM,45.0,New,STREET OPENING PERMIT,118630,124890,2018-01-03,2018,1,4361,360610140001,61,"POLYGON ((-73.95701 40.777544999999996, -73.95...",1
14,M012018004A00,118650124890000000,EAST 83 STREET,LEXINGTON AVENUE,NaN,MANHATTAN,01/09/2018 12:30:54 PM,20.0,Emergency,STREET OPENING PERMIT,118650,124890,2018-01-09,2018,1,4361,360610140001,61,"POLYGON ((-73.95701 40.777544999999996, -73.95...",1
15,M012018008A68,118650124890000000,EAST 83 STREET,LEXINGTON AVENUE,NaN,MANHATTAN,01/09/2018 10:56:11 AM,4.0,Emergency,STREET OPENING PERMIT,118650,124890,2018-01-09,2018,1,4361,360610140001,61,"POLYGON ((-73.95701 40.777544999999996, -73.95...",1
16,M012018022B72,124890118630118656,LEXINGTON AVENUE,EAST 82 STREET,EAST 83 STREET,MANHATTAN,01/24/2018 10:06:38 AM,4.0,Emergency,STREET OPENING PERMIT,124890,118630,2018-01-24,2018,1,4361,360610140001,61,"POLYGON ((-73.95701 40.777544999999996, -73.95...",1


In [ ]:
import pandas as pd

# Convert PermitIssueDate to datetime data type
permits_years['PermitIssueDate'] = pd.to_datetime(permits_years['PermitIssueDate'])

# Extract the week number from the PermitIssueDate
permits_years['Week'] = permits_years['PermitIssueDate'].dt.week

# Group by GEOID, ApplicationTypeShortDesc, and Week, and calculate the count
grouped = permits_years.groupby(['GEOID', 'ApplicationTypeShortDesc']).agg({'PermitLinearFeet': 'sum'}).reset_index()

pivot_table = grouped.pivot(index=['GEOID'], columns='ApplicationTypeShortDesc', values='PermitLinearFeet').fillna(0).reset_index()

pivot_table.head()

ApplicationTypeShortDesc,GEOID,Amend and Reissue,Emergency,Extend,New,Reissue,Renew
0,360050002001,0.0,0.0,0.0,134.0,6.0,0.0
1,360050002002,0.0,406.0,0.0,265.0,6.0,98.0
2,360050002003,460.0,6.0,0.0,934.0,860.0,36.0
3,360050004001,0.0,1211.0,0.0,2791.0,0.0,1666.0
4,360050004002,0.0,0.0,0.0,354.0,0.0,182.0


In [ ]:
pivot_table.shape

(5431, 7)

In [ ]:
bg_nyc_new.head()

,STATEFP,COUNTYFP,TRACTCE,BLKGRPCE,GEOID,NAMELSAD,MTFCC,FUNCSTAT,ALAND,AWATER,INTPTLAT,INTPTLON,geometry
0,36,005,017500,2,360050175002,Block Group 2,G5030,S,23694,0,+40.8296335,-073.9152037,"POLYGON ((1007579.278 241875.151, 1007813.773 ..."
1,36,005,014100,1,360050141001,Block Group 1,G5030,S,55934,0,+40.8245444,-073.9131209,"POLYGON ((1007327.986 239855.027, 1007519.071 ..."
2,36,005,014500,1,360050145001,Block Group 1,G5030,S,47026,0,+40.8309816,-073.9050870,"POLYGON ((1010307.431 242068.478, 1010525.124 ..."
3,36,005,007500,2,360050075002,Block Group 2,G5030,S,109058,0,+40.8202395,-073.9080678,"POLYGON ((1009064.051 238018.664, 1009099.623 ..."
4,36,005,041800,1,360050418001,Block Group 1,G5030,S,140793,0,+40.8989195,-073.8564428,"POLYGON ((1022160.816 265436.445, 1022549.482 ..."


In [ ]:
bg_gas_lengths = pd.merge(pivot_table,bg_nyc_new[["GEOID","geometry"]],on="GEOID")
bg_gas_lenghts = gpd.GeoDataFrame(bg_gas_lengths,geometry="geometry", crs='EPSG:4326')
bg_gas_lenghts.head()

,GEOID,Amend and Reissue,Emergency,Extend,New,Reissue,Renew,geometry
0,360050002001,0.0,0.0,0.0,134.0,6.0,0.0,"POLYGON ((1022080.98608 236153.68321, 1022336...."
1,360050002002,0.0,406.0,0.0,265.0,6.0,98.0,"POLYGON ((1021558.24685 236060.69619, 1021824...."
2,360050002003,460.0,6.0,0.0,934.0,860.0,36.0,"POLYGON ((1023376.66840 233259.63194, 1023472...."
3,360050004001,0.0,1211.0,0.0,2791.0,0.0,1666.0,"POLYGON ((1023113.72261 236337.84900, 1023361...."
4,360050004002,0.0,0.0,0.0,354.0,0.0,182.0,"POLYGON ((1023892.41482 236477.19309, 1024242...."


In [ ]:
bg_gas_lenghts.head()

,GEOID,Amend and Reissue,Emergency,Extend,New,Reissue,Renew,geometry
0,360050002001,0.0,0.0,0.0,134.0,6.0,0.0,"POLYGON ((1022080.98608 236153.68321, 1022336...."
1,360050002002,0.0,406.0,0.0,265.0,6.0,98.0,"POLYGON ((1021558.24685 236060.69619, 1021824...."
2,360050002003,460.0,6.0,0.0,934.0,860.0,36.0,"POLYGON ((1023376.66840 233259.63194, 1023472...."
3,360050004001,0.0,1211.0,0.0,2791.0,0.0,1666.0,"POLYGON ((1023113.72261 236337.84900, 1023361...."
4,360050004002,0.0,0.0,0.0,354.0,0.0,182.0,"POLYGON ((1023892.41482 236477.19309, 1024242...."


In [ ]:
bg_gas_lenghts.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
Int64Index: 5431 entries, 0 to 5430
Data columns (total 8 columns):
 #   Column             Non-Null Count  Dtype   
---  ------             --------------  -----   
 0   GEOID              5431 non-null   object  
 1   Amend and Reissue  5431 non-null   float64 
 2   Emergency          5431 non-null   float64 
 3   Extend             5431 non-null   float64 
 4   New                5431 non-null   float64 
 5   Reissue            5431 non-null   float64 
 6   Renew              5431 non-null   float64 
 7   geometry           5431 non-null   geometry
dtypes: float64(6), geometry(1), object(1)
memory usage: 381.9+ KB


In [ ]:
bg_gas_lenghts.to_file("bg_gas_lenghts.shp")

In [ ]:
gdf = gpd.read_file("bg_gas_lenghts.shp")
gdf.head()

,GEOID,Amend and,Emergency,Extend,New,Reissue,Renew,geometry
0,360050002001,0.0,0.0,0.0,134.0,6.0,0.0,"POLYGON ((1022080.98608 236153.68321, 1022336...."
1,360050002002,0.0,406.0,0.0,265.0,6.0,98.0,"POLYGON ((1021558.24685 236060.69619, 1021824...."
2,360050002003,460.0,6.0,0.0,934.0,860.0,36.0,"POLYGON ((1023376.66840 233259.63194, 1023472...."
3,360050004001,0.0,1211.0,0.0,2791.0,0.0,1666.0,"POLYGON ((1023113.72261 236337.84900, 1023361...."
4,360050004002,0.0,0.0,0.0,354.0,0.0,182.0,"POLYGON ((1023892.41482 236477.19309, 1024242...."


In [ ]:
#sum of linear feet in each BG
import pandas as pd

# Convert PermitIssueDate to datetime data type
permits_years['PermitIssueDate'] = pd.to_datetime(permits_years['PermitIssueDate'])

# Extract the week number from the PermitIssueDate
permits_years['Week'] = permits_years['PermitIssueDate'].dt.week

# Group by GEOID, ApplicationTypeShortDesc, and Week, and calculate the count
grouped = permits_years.groupby(['GEOID', 'ApplicationTypeShortDesc', 'Year','Week']).agg({'PermitLinearFeet': 'sum'}).reset_index()

pivot_table = grouped.pivot(index=['GEOID',"Year",'Week'], columns='ApplicationTypeShortDesc', values='PermitLinearFeet').fillna(0).reset_index()

unique_geoids = pivot_table['GEOID'].unique()
unique_weeks = range(1, 53)
unique_years = pivot_table['Year'].unique()

# Create a new index with all possible combinations of 'GEOID' and 'Week'
new_index = pd.MultiIndex.from_product([unique_geoids,unique_years, unique_weeks], names=['GEOID','Year', 'Week'])

# Set the new index to the DataFrame
pivot_table = pivot_table.set_index(['GEOID','Year','Week'])

# Reindex the DataFrame with the new index and fill missing values with 0
pivot_table = pivot_table.reindex(new_index, fill_value=0).reset_index()

pivot_table.head()

ApplicationTypeShortDesc,GEOID,Year,Week,Amend and Reissue,Emergency,Extend,New,Reissue,Renew
0,360050002001,2019,1,0.0,0.0,0.0,0.0,0.0,0.0
1,360050002001,2019,2,0.0,0.0,0.0,0.0,0.0,0.0
2,360050002001,2019,3,0.0,0.0,0.0,0.0,0.0,0.0
3,360050002001,2019,4,0.0,0.0,0.0,0.0,0.0,0.0
4,360050002001,2019,5,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
pivot_table = pivot_table.rename(columns={"Amend and Reissue":"Amend and Reissue_Length","Emergency":"Emergency_Length",
                                 "Extend":"Extend_Length","New":"New_Length","Renew":"Renew_Length","Reissue":"Reissue_Length"})
pivot_table.head()

ApplicationTypeShortDesc,GEOID,Year,Week,Amend and Reissue_Length,Emergency_Length,Extend_Length,New_Length,Reissue_Length,Renew_Length
0,360050002001,2019,1,0.0,0.0,0.0,0.0,0.0,0.0
1,360050002001,2019,2,0.0,0.0,0.0,0.0,0.0,0.0
2,360050002001,2019,3,0.0,0.0,0.0,0.0,0.0,0.0
3,360050002001,2019,4,0.0,0.0,0.0,0.0,0.0,0.0
4,360050002001,2019,5,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
#sum of linear feet in each BG
import pandas as pd

# Convert PermitIssueDate to datetime data type
permits_years['PermitIssueDate'] = pd.to_datetime(permits_years['PermitIssueDate'])

# Extract the week number from the PermitIssueDate
permits_years['Week'] = permits_years['PermitIssueDate'].dt.week

# Group by GEOID, ApplicationTypeShortDesc, and Week, and calculate the count
grouped_count = permits_years.groupby(['GEOID', 'ApplicationTypeShortDesc', 'Year','Week']).agg({'PermitLinearFeet': 'count'}).reset_index()

pivot_table_count = grouped_count.pivot(index=['GEOID',"Year",'Week'], columns='ApplicationTypeShortDesc', values='PermitLinearFeet').fillna(0).reset_index()

unique_geoids = pivot_table['GEOID'].unique()
unique_weeks = range(1, 53)
unique_years = pivot_table['Year'].unique()

# Create a new index with all possible combinations of 'GEOID' and 'Week'
new_index = pd.MultiIndex.from_product([unique_geoids,unique_years, unique_weeks], names=['GEOID','Year', 'Week'])

# Set the new index to the DataFrame
pivot_table_count = pivot_table_count.set_index(['GEOID','Year','Week'])

# Reindex the DataFrame with the new index and fill missing values with 0
pivot_table_count = pivot_table_count.reindex(new_index, fill_value=0).reset_index()

pivot_table_count.head()

ApplicationTypeShortDesc,GEOID,Year,Week,Amend and Reissue,Emergency,Extend,New,Reissue,Renew
0,360050002001,2019,1,0.0,0.0,0.0,0.0,0.0,0.0
1,360050002001,2019,2,0.0,0.0,0.0,0.0,0.0,0.0
2,360050002001,2019,3,0.0,0.0,0.0,0.0,0.0,0.0
3,360050002001,2019,4,0.0,0.0,0.0,0.0,0.0,0.0
4,360050002001,2019,5,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
pivot_table_count = pivot_table_count.rename(columns={"Amend and Reissue":"Amend and Reissue_Count","Emergency":"Emergency_Count",
                                 "Extend":"Extend_Count","New":"New_Count","Renew":"Renew_Count","Reissue":"Reissue_Count"})
pivot_table_count.head()

ApplicationTypeShortDesc,GEOID,Year,Week,Amend and Reissue_Count,Emergency_Count,Extend_Count,New_Count,Reissue_Count,Renew_Count
0,360050002001,2019,1,0.0,0.0,0.0,0.0,0.0,0.0
1,360050002001,2019,2,0.0,0.0,0.0,0.0,0.0,0.0
2,360050002001,2019,3,0.0,0.0,0.0,0.0,0.0,0.0
3,360050002001,2019,4,0.0,0.0,0.0,0.0,0.0,0.0
4,360050002001,2019,5,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
gas_df = pd.merge(pivot_table,pivot_table_count,on=["GEOID","Year","Week"])
gas_df.head()

ApplicationTypeShortDesc,GEOID,Year,Week,Amend and Reissue_Length,Emergency_Length,Extend_Length,New_Length,Reissue_Length,Renew_Length,Amend and Reissue_Count,Emergency_Count,Extend_Count,New_Count,Reissue_Count,Renew_Count
0,360050002001,2019,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,360050002001,2019,2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,360050002001,2019,3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,360050002001,2019,4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,360050002001,2019,5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
gas_df["Total_Length"] = gas_df["Amend and Reissue_Length"] + gas_df["Emergency_Length"] + gas_df["Extend_Length"] + gas_df["New_Length"] + gas_df["Reissue_Length"] + gas_df["Renew_Length"]
gas_df["Total_Count"] = gas_df["Amend and Reissue_Count"] + gas_df["Emergency_Count"] + gas_df["Extend_Count"] + gas_df["New_Count"] + gas_df["Reissue_Count"] + gas_df["Renew_Count"]

gas_df[gas_df["Year"]==2018]

ApplicationTypeShortDesc,GEOID,Year,Week,Amend and Reissue_Length,Emergency_Length,Extend_Length,New_Length,Reissue_Length,Renew_Length,Amend and Reissue_Count,Emergency_Count,Extend_Count,New_Count,Reissue_Count,Renew_Count,Total_Length,Total_Count
156,360050002001,2018,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
157,360050002001,2018,2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
158,360050002001,2018,3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
159,360050002001,2018,4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
160,360050002001,2018,5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
161,360050002001,2018,6,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
162,360050002001,2018,7,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
163,360050002001,2018,8,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
164,360050002001,2018,9,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
165,360050002001,2018,10,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
gas_df.head()

ApplicationTypeShortDesc,GEOID,Year,Week,Amend and Reissue_Length,Emergency_Length,Extend_Length,New_Length,Reissue_Length,Renew_Length,Amend and Reissue_Count,Emergency_Count,Extend_Count,New_Count,Reissue_Count,Renew_Count,Total_Length,Total_Count
0,360050002001,2019,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,360050002001,2019,2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,360050002001,2019,3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,360050002001,2019,4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,360050002001,2019,5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
#update weeks
gas_df_weeks = gas_df.copy()

def calculate_updated_week(year, week):
    if year == 2018:
        return week
    else:
        return (year - 2018) * 52 + week

# Apply the custom function to update the 'Week' column
gas_df_weeks['Week'] = gas_df_weeks.apply(lambda row: calculate_updated_week(row['Year'], row['Week']), axis=1)
gas_df_weeks.head()

ApplicationTypeShortDesc,GEOID,Year,Week,Amend and Reissue_Length,Emergency_Length,Extend_Length,New_Length,Reissue_Length,Renew_Length,Amend and Reissue_Count,Emergency_Count,Extend_Count,New_Count,Reissue_Count,Renew_Count,Total_Length,Total_Count
0,360050002001,2019,53.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,360050002001,2019,54.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,360050002001,2019,55.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,360050002001,2019,56.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,360050002001,2019,57.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
#gas_df_weeks.to_csv("gas_df_weeks.csv")